In [122]:
import pandas as pd
import numpy as np
import requests
import json
from pandas_ods_reader import read_ods
import pyarrow


In [123]:
# retrieve data from api on the web
url = 'http://chargepoints.dft.gov.uk/api/retrieve/registry/format/csv'
chargepoint_df = pd.read_csv(url)

# COMBINED WITH DATA FROM OCM (OPEN CHARGE MAP)

/var/folders/j9/0qd6cw014pgb7rk0v_28prrh0000gn/T/ipykernel_15876/1205791447.py:3: DtypeWarning: Columns (5,10,11,16,17,18,54,56,57,58,59,60,61,62,63,64,65,66,67,68,69,79,90,101,112,115,119,122,123,126,130,133,134,137,141,144,148,152,155) have mixed types. Specify dtype option on import or set low_memory=False.
  chargepoint_df = pd.read_csv(url)


In [124]:
# get shape of data
chargepoint_df.head()

,chargeDeviceID,reference,name,latitude,longitude,subBuildingName,buildingName,buildingNumber,thoroughfare,street,...,connector8Type,connector8RatedOutputKW,connector8OutputCurrent,connector8RatedVoltage,connector8ChargeMethod,connector8ChargeMode,connector8TetheredCable,connector8Status,connector8Description,connector8Validated
0,c911241d00294e8bb714eee2e83fa475,PP-12289,Alex F Noble & Son,55.875053,-3.173333,NaN,NaN,NaN,NaN,Swinton Place,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,fa6c94460e902005a0b660266190c8ba,PP-12295,Ancaster Nissan Dealership,51.411173,-0.055369,NaN,NaN,61,Croydon Road,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,eb1848290d5a7de9c9ccabc67fefa211,PP-12290,Beadles Nissan Ltd,51.451127,0.050619,NaN,NaN,43-53,Eltham High Street,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,91e50fe1e39af2869d3336eaaeebdb43,PP-12292,Benfield Motors,54.978947,-1.599306,NaN,NaN,176,Portland Road,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,65b1e92c585fd4c2159d5f33b5030ff2,PP-12198,Circus Road,51.533633,-0.172353,NaN,NaN,NaN,Circus Road,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [125]:
chargepoint_df.shape

(31874, 158)

In [126]:
# retreive comments from openchargemap api
url = 'https://api.openchargemap.io/v3/poi/?output=json&countrycode=GB&maxresults=100000&compact=true&verbose=false&key=1e2b0b1e-5b1e-4b3a-8b9a-0b8b9a3a2b1e&includecomments=true'
response = requests.get(url)
data = response.json()

# convert json to dataframe
ocm_comments_df = pd.DataFrame(data)
ocm_comments_df.to_csv('data/ocm_comments.csv')

In [127]:
ocm_comments_df.head()
# unnest the column AddressInfo and UserComments
ocm_comments_df = pd.concat([ocm_comments_df.drop(['AddressInfo'], axis=1), ocm_comments_df['AddressInfo'].apply(pd.Series)], axis=1)

# COMBINED WITH CHARGEPOINT DATA

In [128]:
# join the two dataframes using latitude and longitude
chargepoint_df = chargepoint_df.merge(ocm_comments_df, how='left', left_on=['latitude', 'longitude'], right_on=['Latitude', 'Longitude'])

# COMBINED DATA (OCM + CHARGEPOINT)

In [129]:

# save the data to csv
chargepoint_df.to_csv('data/chargepoint.csv')

chargepoint_df.head()


,chargeDeviceID,reference,name,latitude,longitude,subBuildingName,buildingName,buildingNumber,thoroughfare,street,...,CountryID,Latitude,Longitude,DistanceUnit,AddressLine2,ContactTelephone1,ContactEmail,AccessComments,RelatedURL,ContactTelephone2
0,c911241d00294e8bb714eee2e83fa475,PP-12289,Alex F Noble & Son,55.875053,-3.173333,NaN,NaN,NaN,NaN,Swinton Place,...,1.0,55.875053,-3.173333,0.0,Straiton,0131 440 5353,NaN,Charge Points Located in Customer Parking Area...,http://afnoble.nissan.co.uk/,NaN
1,fa6c94460e902005a0b660266190c8ba,PP-12295,Ancaster Nissan Dealership,51.411173,-0.055369,NaN,NaN,61,Croydon Road,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,eb1848290d5a7de9c9ccabc67fefa211,PP-12290,Beadles Nissan Ltd,51.451127,0.050619,NaN,NaN,43-53,Eltham High Street,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,91e50fe1e39af2869d3336eaaeebdb43,PP-12292,Benfield Motors,54.978947,-1.599306,NaN,NaN,176,Portland Road,NaN,...,1.0,54.978947,-1.599306,0.0,NaN,020 7247 4114,NaN,NaN,NaN,NaN
4,65b1e92c585fd4c2159d5f33b5030ff2,PP-12198,Circus Road,51.533633,-0.172353,NaN,NaN,NaN,Circus Road,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [130]:
# get electricity data with only sheet name "2.2.4" and skipping first 12 rows
electricity_df = pd.read_excel('data/table_224.xlsx', sheet_name='2.2.4', skiprows=12)
# rename the columns
electricity_df.columns = ['Year', 'Region', 'PES area', '1', '2', '3', '4', '5', '6', 'Average variable unit price (£/kWh)', '7']
# drop columns 1-7
electricity_df = electricity_df.drop(['1', '2', '3', '4', '5', '6', '7'], axis=1)
# drop Region column
electricity_df = electricity_df.drop(['Region'], axis=1)
electricity_df.head()

# COMBINED WITH FUEL PRICES

,Year,PES area,Average variable unit price (£/kWh)
0,2010,Northern Scotland,0.118306
1,2010,Northern Ireland,0.147227
2,2010,West Midlands,0.113377
3,2010,South East,0.109783
4,2010,South Wales,0.122350


In [131]:
# show unique values in PES area
electricity_df['PES area'].unique()

array(['Northern Scotland', 'Northern Ireland', 'West Midlands',
       'South East', 'South Wales', 'Southern Scotland', 'Eastern',
       'Yorkshire', 'Merseyside & North Wales', 'London', 'North West',
       'North East', 'East Midlands', 'South West', 'Southern',
       'United Kingdom', 'North Scotland', 'South Scotland'], dtype=object)

In [132]:
# replace PES area values with the correct names

electricity_df['PES area'] = electricity_df['PES area'].replace(['Northern Scotland'], 'North Scotland')
electricity_df['PES area'] = electricity_df['PES area'].replace(['Southern Scotland'], 'South Scotland')
# check if the values are replaced
electricity_df['PES area'].unique()

array(['North Scotland', 'Northern Ireland', 'West Midlands',
       'South East', 'South Wales', 'South Scotland', 'Eastern',
       'Yorkshire', 'Merseyside & North Wales', 'London', 'North West',
       'North East', 'East Midlands', 'South West', 'Southern',
       'United Kingdom'], dtype=object)

In [133]:
# gdhi data
gdhi_df = pd.read_excel("data/regionalgrossdisposablehouseholdincomeallitlregions.xls", sheet_name="Table 3", skiprows=1)
# melt the dataframe
gdhi_df = pd.melt(gdhi_df, id_vars=['Region name', 'ITL level','ITL code'], var_name='Year', value_vars=['1997', '1998', 
                                                                                                         '1999', '2000', '2001', '2002', 
                                                                                                         '2003','2004', '2005', '2006', 
                                                                                                         '2007', '2008', '2009', '2010',
                                                                                                         '2011', '2012', '2013', '2014', 
                                                                                                         '2015', '2016', '2017','2018', 
                                                                                                         '2019', '2020'])
# rename the column value to gdhi
gdhi_df = gdhi_df.rename(columns={'value': 'gdhi'})
gdhi_df.head()

# COMBINED WITH POPULATION DATA

,Region name,ITL level,ITL code,Year,gdhi
0,United Kingdom,UK,UK,1997,10757
1,England,Other,TLB,1997,11016
2,North East,ITL1,TLC,1997,9253
3,Tees Valley and Durham,ITL2,TLC1,1997,9200
4,Hartlepool and Stockton-on-Tees,ITL3,TLC11,1997,9264


In [134]:
# population data
population_df = pd.read_csv('data/ukdetailedtimeseries2001to2020/MYEB1_detailed_population_estimates_series_UK_(2020_geog21).csv')
# remove word "population" from column names
population_df.columns = population_df.columns.str.replace('population_', '')

# melt the dataframe
population_df = pd.melt(population_df, id_vars=['ladcode21', 'laname21','country','sex','age'], var_name='Year', 
                        value_vars=['2001', '2002', '2003', '2004', '2005', '2006', '2007','2008', '2009', '2010', '2011', '2012', 
                                    '2013','2014', '2015', '2016', '2017', '2018', '2019', '2020'])
# rename the column value to population
population_df = population_df.rename(columns={'value': 'population'})

# remove ladcode21 column
population_df = population_df.drop(['ladcode21','country'], axis=1)

# COMBINED WITH GDHI DATA



In [135]:
# correct the values in laname21 column to match the values in gdhi data "Region name" columns
# change Barking and Dagenham to "Barking & Dagenham and Havering"
population_df['laname21'] = population_df['laname21'].replace(['Barking and Dagenham', 'Havering'], 'Barking & Dagenham and Havering')
# change "Angus" to "Angus and Dundee City"
population_df['laname21'] = population_df['laname21'].replace(['Angus', 'Dundee City'], 'Angus and Dundee City')
# change Maldon to Heart of Essex
population_df.loc[population_df['laname21'].str.contains('Maldon', case=False, na=False), 'laname21'] = 'Heart of Essex'
# change the values in laname21 column to match the values in gdhi data "Region name" columns
# change "Hartlepool" to "Durham CC"
population_df['laname21'] = population_df['laname21'].replace(['Hartlepool'], 'Hartlepool and Stockton-on-Tees')
# change "Aberdeen City" to "Aberdeen City and Aberdeenshire"
population_df['laname21'] = population_df['laname21'].replace(['Aberdeen City'], 'Aberdeen City and Aberdeenshire')
# change "Aberdeenshire" to "Aberdeen City and Aberdeenshire"
population_df['laname21'] = population_df['laname21'].replace(['Aberdeenshire'], 'Aberdeen City and Aberdeenshire')
# change "Adur" to "West Sussex (South West)"
population_df['laname21'] = population_df['laname21'].replace(['Worthing', 'Adur', 'Arun', 'Chichester', 'Horsham', 'Mid Sussex'], 'West Sussex (South West)')
# change "Allerdale" to "West Cumbria"
population_df['laname21'] = population_df['laname21'].replace(['Allerdale', 'Barrow-in-Furness', 'Carlisle', 'Copeland'], 'West Cumbria')
# change "Allerdale" to "West Cumbria"
population_df['laname21'] = population_df['laname21'].replace(['Eden', 'South Lakeland'], 'East Cumbria')
# change "Argyll and Bute" to "Lochaber, Skye and Lochalsh, Arran and Cumbrae and Argyll and Bute"
population_df['laname21'] = population_df['laname21'].replace(['Argyll and Bute','Argyl and Bute'], 'Lochaber, Skye and Lochalsh, Arran and Cumbrae and Argyll and Bute')
# change Ashfield to "North Nottinghamshire"
population_df['laname21'] = population_df['laname21'].replace(['Ashfield', 'Bassetlaw', 'Bolsover', 'Gedling'], 'North Nottinghamshire')
# change Babergh to "Suffolk"
population_df['laname21'] = population_df['laname21'].replace(['Babergh', 'East Suffolk', 'Ipswich', 'Mid Suffolk', 'West Suffolk'], 'Suffolk')
# change Barnsley to "Barnsley, Doncaster and Rotherham"
population_df['laname21'] = population_df['laname21'].replace(['Barnsley', 'Doncaster', 'Rotherham'], 'Barnsley, Doncaster and Rotherham')
# change Basildon to "Thurrock"
population_df['laname21'] = population_df['laname21'].replace(['Basildon'], 'Thurrock')
# change Basingstoke and Deane to "North Hampshire"
population_df['laname21'] = population_df['laname21'].replace(['Basingstoke and Deane', 'Hart', 'Rushmoor'], 'North Hampshire')
# change Bath and North East Somerset to "Bath and North East Somerset, North Somerset and South Gloucestershire"
population_df['laname21'] = population_df['laname21'].replace(['Bath and North East Somerset', 'North Somerset', 'South Gloucestershire'], 'Bath and North East Somerset, North Somerset and South Gloucestershire')
# change Bexley to "Bexley and Greenwich"
population_df['laname21'] = population_df['laname21'].replace(['Bexley', 'Greenwich'], 'Bexley and Greenwich')
# change Blaby to "Leicestershire CC and Rutland"
population_df['laname21'] = population_df['laname21'].replace(['Blaby', 'Charnwood', 'Harborough', 'Hinckley and Bosworth', 'Melton', 'North West Leicestershire', 'Oadby and Wigston', 'Rutland'], 'Leicestershire CC and Rutland')
# change Blaenau Gwent to "Gwent Valleys"
population_df['laname21'] = population_df['laname21'].replace(['Blaenau Gwent', 'Torfaen'], 'Gwent Valleys')
# change Bolton to "Greater Manchester North West"
population_df['laname21'] = population_df['laname21'].replace(['Bolton', 'Bury'], 'Greater Manchester North West')
# change Boston to "Lincolnshire"
population_df['laname21'] = population_df['laname21'].replace(['Boston', 'North Kesteven', 'West Lindsey', 'South Kesteven', 'South Holland'], 'Lincolnshire')
# change Bracknell Forest to "Berkshire"
population_df['laname21'] = population_df['laname21'].replace(['Bracknell Forest', 'Reading', 'Slough', 'West Berkshire', 'Windsor and Maidenhead', 'Wokingham'], 'Berkshire')
# change Braintree to "Essex Haven Gateway"
population_df['laname21'] = population_df['laname21'].replace(['Braintree', 'Brentwood', 'Castle Point', 'Colchester', 'Chelmsford', 'Rochford'], 'Essex Haven Gateway')
# change Breckland to "Breckland and South Norfolk"
population_df['laname21'] = population_df['laname21'].replace(['Breckland', 'Broadland', 'Great Yarmouth', 'South Norfolk'], 'Breckland and South Norfolk')
# change Bridgend to "Bridgend and Neath Port Talbot"
population_df['laname21'] = population_df['laname21'].replace(['Bridgend', 'Merthyr Tydfil', 'Neath Port Talbot', 'Rhondda Cynon Taf'], 'Bridgend and Neath Port Talbot')
#  change Bromsgrove to "Worcestershire"
population_df['laname21'] = population_df['laname21'].replace(['Bromsgrove', 'Malvern Hills', 'Redditch', 'Worcester', 'Wychavon'], 'Worcestershire')
# change Broxbourne to "Hertfordshire"
population_df['laname21'] = population_df['laname21'].replace(['Watford', 'Stevenage', 'St Albans', 'Three Rivers', 'Welwyn Hatfield', 'North Hertfordshire', 'Broxbourne', 'East Hertfordshire', 'Dacorum', 'Hertsmere'], 'Hertfordshire')
# change Broxtowe to "South and West Derbyshire"
population_df['laname21'] = population_df['laname21'].replace(['Broxtowe', 'Amber Valley', 'Derbyshire Dales', 'Erewash', 'South Derbyshire'], 'South and West Derbyshire')
# change Buckinghamshire to "Buckinghamshire CC"
population_df['laname21'] = population_df['laname21'].replace(['Buckinghamshire'], 'Buckinghamshire CC')
# change Burnley to "East Lancashire"
population_df['laname21'] = population_df['laname21'].replace(['Burnley', 'Hyndburn', 'Pendle', 'Ribble Valley', 'Rossendale'], 'East Lancashire')
# change Caerphilly to "Cardiff and Vale of Glamorgan"
population_df['laname21'] = population_df['laname21'].replace(['Caerphilly', 'Cardiff', 'Vale of Glamorgan'], 'Cardiff and Vale of Glamorgan')
# change Cambridge to Cambridgeshire CC
population_df['laname21'] = population_df['laname21'].replace(['Cambridge', 'East Cambridgeshire', 'Fenland', 'Huntingdonshire', 'South Cambridgeshire', 'Cambridgeshire'], 'Cambridgeshire CC')
# change Cannock Chase to Staffordshire CC
population_df['laname21'] = population_df['laname21'].replace(['Newcastle-under-Lyme', 'Cannock Chase', 'East Staffordshire', 'Lichfield', 'South Staffordshire', 'Tamworth', 'Staffordshire Moorlands', 'Stafford'], 'Staffordshire CC')
# change Carmarthenshire to South West Wales
population_df['laname21'] = population_df['laname21'].replace(['Carmarthenshire', 'Pembrokeshire'], 'South West Wales')
# change Ceredigion to Central Valleys
population_df.loc[population_df['laname21'].str.contains('Ceredigion', case=False, na=False), 'laname21'] = 'Central Valleys'
# change Cherwell to Oxfordshire
population_df['laname21'] = population_df['laname21'].replace(['Cherwell', 'Oxford', 'South Oxfordshire', 'Vale of White Horse', 'West Oxfordshire'], 'Oxfordshire')
# change Chesterfield to East Derbyshire
population_df['laname21'] = population_df['laname21'].replace(['Chesterfield', 'North East Derbyshire', 'High Peak'], 'East Derbyshire')
# change Chorley to Chorley and West Lancashire
population_df['laname21'] = population_df['laname21'].replace(['Chorley', 'Fylde', 'South Ribble', 'West Lancashire'], 'Chorley and West Lancashire')
# change City of London to Camden and City of London
population_df['laname21'] = population_df['laname21'].replace(['City of London', 'Camden'], 'Camden and City of London')
# change Clackmannanshire to Clackmannanshire and Fife
population_df['laname21'] = population_df['laname21'].replace(['Clackmannanshire', 'Fife'], 'Clackmannanshire and Fife')
# change Conwy to Conwy and Denbighshire
population_df['laname21'] = population_df['laname21'].replace(['Conwy', 'Denbighshire'], 'Conwy and Denbighshire')
# change Cornwall to Cornwall and Isles of Scilly
population_df['laname21'] = population_df['laname21'].replace(['Cornwall', 'Isles of Scilly'], 'Cornwall and Isles of Scilly')
# change County Durham to Durham CC
population_df.loc[population_df['laname21'].str.contains('County Durham', case=False, na=False), 'laname21'] = 'Durham CC'
# change Crawley to West Sussex
population_df.loc[population_df['laname21'].str.contains('Crawley', case=False, na=False), 'laname21'] = 'West Sussex (North East)'
# change East Ayrshire to East Ayrshire and North Ayrshire mainland
population_df['laname21'] = population_df['laname21'].replace(['East Ayrshire', 'North Ayrshire'], 'East Ayrshire and North Ayrshire mainland')
# change East Devon to Devon CC
population_df['laname21'] = population_df['laname21'].replace(['East Devon', 'Exeter', 'Mid Devon', 'North Devon', 'South Hams', 'Teignbridge', 'Torridge', 'West Devon'], 'Devon CC')
# change East Dunbartonshire to East Dunbartonshire, West Dunbartonshire and Helensburgh and Lomond
population_df['laname21'] = population_df['laname21'].replace(['East Dunbartonshire', 'West dunbartonshire', 'West Dunbartonshire'], 'East Dunbartonshire, West Dunbartonshire and Helensburgh and Lomond')
# change East Hampshire to Central Hampshire
population_df['laname21'] = population_df['laname21'].replace(['East Hampshire', 'Eastleigh', 'New Forest'], 'Central Hampshire')
# change East Lothian to East Lothian and Midlothian
population_df['laname21'] = population_df['laname21'].replace(['East Lothian', 'Midlothian'], 'East Lothian and Midlothian')
# change East Renfrewshire to Inverclyde, East Renfrewshire and Renfrewshire
population_df['laname21'] = population_df['laname21'].replace(['East Renfrewshire', 'Inverclyde', 'Renfrewshire'], 'Inverclyde, East Renfrewshire and Renfrewshire')
# change Eastbourne to East Sussex
population_df['laname21'] = population_df['laname21'].replace(['Eastbourne', 'East Sussex'])
# change Elmbridge to West Surrey
population_df['laname21'] = population_df['laname21'].replace(['Elmbridge', 'Epsom and Ewell', 'Guildford', 'Runnymede', 'Spelthorne', 'Woking', 'Waverley', 'Surrey Heath'], 'West Surrey')
# change Fareham to South Hampshire
population_df['laname21'] = population_df['laname21'].replace(['Fareham', 'Gosport', 'Havant'], 'South Hampshire')
# change Flintshire to Flintshire and Wrexham
population_df['laname21'] = population_df['laname21'].replace(['Flintshire', 'Wrexham'], 'Flintshire and Wrexham')
# change Folkestone and Hythe to East Kent
population_df['laname21'] = population_df['laname21'].replace(['Folkestone and Hythe', 'Dover', 'Ashford', 'Canterbury'], 'East Kent')
# change Forest of Dean to Gloucestershire
population_df['laname21'] = population_df['laname21'].replace(['Gloucester', 'Forest of Dean', 'Cotswold', 'Cheltenham', 'Stroud', 'Tewkesbury'], 'Gloucestershire')
# change Gateshead to Tyneside
population_df['laname21'] = population_df['laname21'].replace(['Newcastle upon Tyne', 'Gateshead', 'North Tyneside', 'South Tyneside'], 'Tyneside')
# change Hackney to Hackney and Newham
population_df['laname21'] = population_df['laname21'].replace(['Hackney', 'Newham'], 'Hackney and Newham')
# change Hammersmith and Fulham to Kensington & Chelsea and Hammersmith & Fulham
population_df['laname21'] = population_df['laname21'].replace(['Hammersmith and Fulham', 'Kensington and Chelsea'], 'Kensington & Chelsea and Hammersmith & Fulham')
# change Harlow to Essex Thames Gateway
population_df['laname21'] = population_df['laname21'].replace(['Harlow', 'Epping Forest'], 'Essex Thames Gateway')
# change Harrow to Harrow and Hillingdon
population_df['laname21'] = population_df['laname21'].replace(['Harrow', 'Hillingdon'], 'Harrow and Hillingdon')
# change Haringey to Haringey and Islington
population_df['laname21'] = population_df['laname21'].replace(['Haringey', 'Islington'], 'Haringey and Islington')
# change Hastings to East Sussex CC
population_df['laname21'] = population_df['laname21'].replace(['Hastings', 'Lewes', 'Rother', 'Wealden'], 'East Sussex CC')
# change Highland to Highlands and Islands
population_df.loc[population_df['laname21'].str.contains('Highland', case=False, na=False), 'laname21'] = 'Caithness and Sutherland and Ross and Cromarty'
# change Hounslow to Hounslow and Richmond upon Thames
population_df['laname21'] = population_df['laname21'].replace(['Hounslow', 'Richmond upon Thames'], 'Hounslow and Richmond upon Thames')
# change King's Lynn and West Norfolk to North and West Norfolk
population_df['laname21'] = population_df['laname21'].replace(["King's Lynn and West Norfolk", 'North Norfolk'], 'North and West Norfolk')
# change Kingston upon Thames to Merton, Kingston upon Thames and Sutton
population_df['laname21'] = population_df['laname21'].replace(['Kingston upon Thames', 'Merton', 'Sutton'], 'Merton, Kingston upon Thames and Sutton')
# change Kirklees to Calderdale and Kirklees
population_df['laname21'] = population_df['laname21'].replace(['Kirklees', 'Calderdale'], 'Calderdale and Kirklees')
# change Knowsley to East Merseyside
population_df['laname21'] = population_df['laname21'].replace(['Knowsley', 'St. Helens'], 'East Merseyside')
# change Lancaster to Lancaster and Wyre
population_df['laname21'] = population_df['laname21'].replace(['Lancaster', 'Wyre', 'Wyre Forest'], 'Lancaster and Wyre')
# change Lewisham to Lewisham and Southwark
population_df['laname21'] = population_df['laname21'].replace(['Lewisham', 'Southwark'], 'Lewisham and Southwark')
# change Lincoln to North and North East Lincolnshire
population_df['laname21'] = population_df['laname21'].replace(['Lincoln', 'North East Lincolnshire', 'North Lincolnshire', 'East Lindsey'], 'North and North East Lincolnshire')
# change Maidstone to Medway
population_df['laname21'] = population_df['laname21'].replace('Maidstone', 'Medway')
# change Mansfield to Nottingham
population_df['laname21'] = population_df['laname21'].replace(['Mansfield', 'Newark and Sherwood'], 'Nottingham')
# change Mendip to Somerset
population_df['laname21'] = population_df['laname21'].replace(['Mendip', 'South Somerset', 'Somerset West and Taunton', 'Sedgemoor'], 'Somerset')
# change Mole Valley to East Surrey
population_df['laname21'] = population_df['laname21'].replace(['Mole Valley', 'Reigate and Banstead', 'Tandridge'], 'East Surrey')
# change Monmouthshire to Monmouthshire and Newport
population_df['laname21'] = population_df['laname21'].replace(['Monmouthshire', 'Newport'], 'Monmouthshire and Newport')
# change Moray to Inverness and Nairn and Moray, Badenoch and Strathspey
population_df['laname21'] = population_df['laname21'].replace(['Inverness-Shire','Moray'], 'Inverness and Nairn and Moray, Badenoch and Strathspey')
# change North Warwickshire to Warwickshire
population_df['laname21'] = population_df['laname21'].replace(['Warwick', 'North Warwickshire', 'Nuneaton and Bedworth', 'Rugby', 'Stratford-on-Avon'], 'Warwickshire')
# change Norwich to Norwich and East Norfolk
population_df.loc[population_df['laname21'].str.contains('Norwich', case=False, na=False), 'laname21'] = 'Norwich and East Norfolk'
# change Oldham to Greater Manchester North East
population_df['laname21'] = population_df['laname21'].replace(['Oldham', 'Rochdale', 'Salford'], 'Greater Manchester North East')
# change Perth and Kinross to Perth and Kinross and Stirling
population_df['laname21'] = population_df['laname21'].replace(['Perth and Kinross', 'Stirling'], 'Perth and Kinross and Stirling')
# change preston to Mid Lancashire
population_df.loc[population_df['laname21'].str.contains('Preston', case=False, na=False), 'laname21'] = 'Mid Lancashire'
# change 'Redcar and Cleveland' to South Teesside
population_df['laname21'] = population_df['laname21'].replace(['Redcar and Cleveland'], 'South Teesside')
# change Redcar and Cleveland to North Yorkshire CC
population_df['laname21'] = population_df['laname21'].replace(['Richmondshire', 'Middlesbrough', 'Hambleton', 'Harrogate', 'Ryedale', 'Scarborough', 'Selby', 'Craven'], 'North Yorkshire CC')
# change Redbridge to Redbridge and Waltham Forest
population_df['laname21'] = population_df['laname21'].replace(['Redbridge', 'Waltham Forest'], 'Redbridge and Waltham Forest')
# change Rushcliffe to Nottinghamshire
population_df.loc[population_df['laname21'].str.contains('Rushcliffe', case=False, na=False), 'laname21'] = 'South Nottinghamshire'
# change Shropshire to Shropshire CC
population_df.loc[population_df['laname21'].str.contains('Shropshire', case=False, na=False), 'laname21'] = 'Shropshire CC'
# change Stockport to Greater Manchester South East
population_df['laname21'] = population_df['laname21'].replace(['Stockport', 'Tameside', 'Wigan'], 'Greater Manchester South East')
# change 'Trafford' to Greater Manchester South West
population_df['laname21'] = population_df['laname21'].replace(['Trafford'], 'Greater Manchester South West')
# change Stockton-on-Tees to Hartlepool and Stockton-on-Tees
population_df['laname21'] = population_df['laname21'].replace(['Stockton-on-Tees', 'Hartlepool and Stockton-on-Tees'])
# change Tendring to Essex
population_df['laname21'] = population_df['laname21'].replace(['Tendring', 'Uttlesford'], 'West Essex')
# change Test Valley to Central Hampshire
population_df['laname21'] = population_df['laname21'].replace(['Test Valley'], 'Central Hampshire')
# change Winchester to Portsmouth
population_df.loc[population_df['laname21'].str.contains('Winchester', case=False, na=False), 'laname21'] = 'Portsmouth'
# 'Tonbridge and Malling' to west Kent
population_df['laname21'] = population_df['laname21'].replace(['Tonbridge and Malling', 'Tunbridge Wells', 'Sevenoaks'], 'West Kent')
# change Tunbridge Wells to Kent Thames Gateway
population_df['laname21'] = population_df['laname21'].replace(['Thanet', 'Swale', 'Gravesham', 'Dartford'], 'Kent Thames Gateway')

# group the population data by year, laname21, sex and age and sum the population
population_df = population_df.groupby(['Year', 'laname21', 'sex', 'age'])['population'].sum().reset_index()

# check if population data and gdhi data have the same laname21 and Region name and save as text file
set_pop = set(gdhi_df['Region name'].unique()) - set(population_df['laname21'].unique())


In [136]:
# merge the dataframes gdhi_df and population_df on laname21 and Year
gdhi_population_df = gdhi_df.merge(population_df, how='right', left_on=['Region name', 'Year'], right_on=['laname21', 'Year'])
gdhi_population_df

# rename Region name to ITL level 3
gdhi_population_df = gdhi_population_df.rename(columns={'Region name': 'ITL level 3'})

# drop laname21 column
gdhi_population_df = gdhi_population_df.drop(['laname21'], axis=1)

# remove all rows with ITL2
gdhi_population_df = gdhi_population_df[gdhi_population_df['ITL level'] == 'ITL3']

# Create a new column ITL level 1 such that it is the first 3 letters of ITL code are TLC then value is North East, 
# if TLD then value is North West, 
# if TLE then value is Yorkshire and The Humber, 
# if TLF then value is East Midlands, 
# if TLG then value is West Midlands, 
# if TLH then value is East of England, 
# if TLI then value is London, 
# if TLJ then value is South East, 
# if TLK then value is South West, 
# if TLM then value is Wales, 
# if TLN then value is Scotland, 
# if TLP then value is Northern Ireland
gdhi_population_df['ITL level 1'] = gdhi_population_df['ITL code'].str[:3]
gdhi_population_df['ITL level 1'] = gdhi_population_df['ITL level 1'].replace(['TLC'], 'North East')
gdhi_population_df['ITL level 1'] = gdhi_population_df['ITL level 1'].replace(['TLD'], 'North West')
gdhi_population_df['ITL level 1'] = gdhi_population_df['ITL level 1'].replace(['TLE'], 'Yorkshire and The Humber')
gdhi_population_df['ITL level 1'] = gdhi_population_df['ITL level 1'].replace(['TLF'], 'East Midlands')
gdhi_population_df['ITL level 1'] = gdhi_population_df['ITL level 1'].replace(['TLG'], 'West Midlands')
gdhi_population_df['ITL level 1'] = gdhi_population_df['ITL level 1'].replace(['TLH'], 'East of England')
gdhi_population_df['ITL level 1'] = gdhi_population_df['ITL level 1'].replace(['TLI'], 'London')
gdhi_population_df['ITL level 1'] = gdhi_population_df['ITL level 1'].replace(['TLJ'], 'South East')
gdhi_population_df['ITL level 1'] = gdhi_population_df['ITL level 1'].replace(['TLK'], 'South West')
gdhi_population_df['ITL level 1'] = gdhi_population_df['ITL level 1'].replace(['TLL'], 'Wales')
gdhi_population_df['ITL level 1'] = gdhi_population_df['ITL level 1'].replace(['TLM'], 'Scotland')
gdhi_population_df['ITL level 1'] = gdhi_population_df['ITL level 1'].replace(['TLN'], 'Northern Ireland')

# create a new column ITL level 2 such that it is the first 4 letters of ITL code are TLC1 then value is Tees Valley and Durham,
# if TLC2 then value is Northumberland and Tyne and Wear,
# if TLD1 then value is Cumbria,
# if TLD3 then value is Greater Manchester,
# if TLD4 then value is Lancashire,
# if TLD6 then value is Cheshire,
# if TLD7 then value is Merseyside,
# if TLE1 then value is East Yorkshire and Northern Lincolnshire,
# if TLE2 then value is North Yorkshire,
# if TLE3 then value is South Yorkshire,
# if TLE4 then value is West Yorkshire,
# if TLF1 then value is Derbyshire and Nottinghamshire,
# if TLF2 then value is Leicestershire, Rutland and Northamptonshire,
# if TLF3 then value is Lincolnshire,
# if TLG1 then value is Herefordshire, Worcestershire and Warwickshire,
# if TLG2 then value is Shropshire and Staffordshire,
# if TLG3 then value is West Midlands,
# if TLH1 then value is East Anglia,
# if TLH2 then value is Bedfordshire and Hertfordshire,
# if TLH3 then value is Essex,
# if TLI3 then value is Inner London - West,
# if TLI4 then value is Inner London - East,
# if TLI5 then value is Outer London - East and North East,
# if TLI6 then value is Outer London - South,
# if TLI7 then value is Outer London - West and North West,
# if TLJ1 then value is Berkshire, Buckinghamshire and Oxfordshire,
# if TLJ2 then value is Surrey, East and West Sussex,
# if TLJ3 then value is Hampshire and Isle of Wight,
# if TLJ4 then value is Kent,
# if TLK1 then value is Gloucestershire, Wiltshire and Bristol/Bath area,
# if TLK2 then value is Dorset and Somerset,
# if TLK3 then value is Cornwall and Isles of Scilly,
# if TLK4 then value is Devon,
# if TLL1 then value is West Wales and The Valleys,
# if TLL2 then value is East Wales,
# if TLM5 then value is North Eastern Scotland,
# if TLM6 then value is Highlands and Islands,
# if TLM7 then value is Eastern Scotland,
# if TLM8 then value is West Central Scotland,
# if TLM9 then value is Southern Scotland,
# if TLN0 then value is Northern Ireland
gdhi_population_df['ITL level 2'] = gdhi_population_df['ITL code'].str[:4]
gdhi_population_df['ITL level 2'] = gdhi_population_df['ITL level 2'].replace(['TLC1'], 'Tees Valley and Durham')
gdhi_population_df['ITL level 2'] = gdhi_population_df['ITL level 2'].replace(['TLC2'], 'Northumberland and Tyne and Wear')
gdhi_population_df['ITL level 2'] = gdhi_population_df['ITL level 2'].replace(['TLD1'], 'Cumbria')
gdhi_population_df['ITL level 2'] = gdhi_population_df['ITL level 2'].replace(['TLD3'], 'Greater Manchester')
gdhi_population_df['ITL level 2'] = gdhi_population_df['ITL level 2'].replace(['TLD4'], 'Lancashire')
gdhi_population_df['ITL level 2'] = gdhi_population_df['ITL level 2'].replace(['TLD6'], 'Cheshire')
gdhi_population_df['ITL level 2'] = gdhi_population_df['ITL level 2'].replace(['TLD7'], 'Merseyside')
gdhi_population_df['ITL level 2'] = gdhi_population_df['ITL level 2'].replace(['TLE1'], 'East Yorkshire and Northern Lincolnshire')
gdhi_population_df['ITL level 2'] = gdhi_population_df['ITL level 2'].replace(['TLE2'], 'North Yorkshire')
gdhi_population_df['ITL level 2'] = gdhi_population_df['ITL level 2'].replace(['TLE3'], 'South Yorkshire')
gdhi_population_df['ITL level 2'] = gdhi_population_df['ITL level 2'].replace(['TLE4'], 'West Yorkshire')
gdhi_population_df['ITL level 2'] = gdhi_population_df['ITL level 2'].replace(['TLF1'], 'Derbyshire and Nottinghamshire')
gdhi_population_df['ITL level 2'] = gdhi_population_df['ITL level 2'].replace(['TLF2'], 'Leicestershire, Rutland and Northamptonshire')
gdhi_population_df['ITL level 2'] = gdhi_population_df['ITL level 2'].replace(['TLF3'], 'Lincolnshire')
gdhi_population_df['ITL level 2'] = gdhi_population_df['ITL level 2'].replace(['TLG1'], 'Herefordshire, Worcestershire and Warwickshire')
gdhi_population_df['ITL level 2'] = gdhi_population_df['ITL level 2'].replace(['TLG2'], 'Shropshire and Staffordshire')
gdhi_population_df['ITL level 2'] = gdhi_population_df['ITL level 2'].replace(['TLG3'], 'West Midlands')
gdhi_population_df['ITL level 2'] = gdhi_population_df['ITL level 2'].replace(['TLH1'], 'East Anglia')
gdhi_population_df['ITL level 2'] = gdhi_population_df['ITL level 2'].replace(['TLH2'], 'Bedfordshire and Hertfordshire')
gdhi_population_df['ITL level 2'] = gdhi_population_df['ITL level 2'].replace(['TLH3'], 'Essex')
gdhi_population_df['ITL level 2'] = gdhi_population_df['ITL level 2'].replace(['TLI3'], 'Inner London - West')
gdhi_population_df['ITL level 2'] = gdhi_population_df['ITL level 2'].replace(['TLI4'], 'Inner London - East')
gdhi_population_df['ITL level 2'] = gdhi_population_df['ITL level 2'].replace(['TLI5'], 'Outer London - East and North East')
gdhi_population_df['ITL level 2'] = gdhi_population_df['ITL level 2'].replace(['TLI6'], 'Outer London - South')
gdhi_population_df['ITL level 2'] = gdhi_population_df['ITL level 2'].replace(['TLI7'], 'Outer London - West and North West')
gdhi_population_df['ITL level 2'] = gdhi_population_df['ITL level 2'].replace(['TLJ1'], 'Berkshire, Buckinghamshire and Oxfordshire')
gdhi_population_df['ITL level 2'] = gdhi_population_df['ITL level 2'].replace(['TLJ2'], 'Surrey, East and West Sussex')
gdhi_population_df['ITL level 2'] = gdhi_population_df['ITL level 2'].replace(['TLJ3'], 'Hampshire and Isle of Wight')
gdhi_population_df['ITL level 2'] = gdhi_population_df['ITL level 2'].replace(['TLJ4'], 'Kent')
gdhi_population_df['ITL level 2'] = gdhi_population_df['ITL level 2'].replace(['TLK1'], 'Gloucestershire, Wiltshire and Bristol/Bath area')
gdhi_population_df['ITL level 2'] = gdhi_population_df['ITL level 2'].replace(['TLK2'], 'Dorset and Somerset')
gdhi_population_df['ITL level 2'] = gdhi_population_df['ITL level 2'].replace(['TLK3'], 'Cornwall and Isles of Scilly')
gdhi_population_df['ITL level 2'] = gdhi_population_df['ITL level 2'].replace(['TLK4'], 'Devon')
gdhi_population_df['ITL level 2'] = gdhi_population_df['ITL level 2'].replace(['TLL1'], 'West Wales and The Valleys')
gdhi_population_df['ITL level 2'] = gdhi_population_df['ITL level 2'].replace(['TLL2'], 'East Wales')
gdhi_population_df['ITL level 2'] = gdhi_population_df['ITL level 2'].replace(['TLM5'], 'North Eastern Scotland')
gdhi_population_df['ITL level 2'] = gdhi_population_df['ITL level 2'].replace(['TLM6'], 'Highlands and Islands')
gdhi_population_df['ITL level 2'] = gdhi_population_df['ITL level 2'].replace(['TLM7'], 'Eastern Scotland')
gdhi_population_df['ITL level 2'] = gdhi_population_df['ITL level 2'].replace(['TLM8'], 'West Central Scotland')
gdhi_population_df['ITL level 2'] = gdhi_population_df['ITL level 2'].replace(['TLM9'], 'Southern Scotland')
gdhi_population_df['ITL level 2'] = gdhi_population_df['ITL level 2'].replace(['TLN0'], 'Northern Ireland')


gdhi_population_df.head()

# COMBINED GDHI AND POPULATION DATA

,ITL level 3,ITL level,ITL code,Year,gdhi,sex,age,population,ITL level 1,ITL level 2
0,Aberdeen City and Aberdeenshire,ITL3,TLM50,2001,12622.0,1,0,2267,Scotland,North Eastern Scotland
1,Aberdeen City and Aberdeenshire,ITL3,TLM50,2001,12622.0,1,1,2373,Scotland,North Eastern Scotland
2,Aberdeen City and Aberdeenshire,ITL3,TLM50,2001,12622.0,1,2,2445,Scotland,North Eastern Scotland
3,Aberdeen City and Aberdeenshire,ITL3,TLM50,2001,12622.0,1,3,2481,Scotland,North Eastern Scotland
4,Aberdeen City and Aberdeenshire,ITL3,TLM50,2001,12622.0,1,4,2569,Scotland,North Eastern Scotland


In [137]:
# show unique values in ITL level 1
gdhi_population_df['ITL level 1'].unique()

array(['Scotland', 'Northern Ireland', 'London',
       'Yorkshire and The Humber', 'South West', 'East of England',
       'South East', 'West Midlands', 'North West', 'Wales', 'North East',
       'East Midlands'], dtype=object)

In [138]:
path = "data/veh0105.ods"

# load a sheet based on its index (1 based)
sheet_idx = 4
vehicle_fuel_df = read_ods(path, sheet_idx)

# make third row the header
vehicle_fuel_df.columns = vehicle_fuel_df.iloc[3]
# drop first 4 rows
vehicle_fuel_df = vehicle_fuel_df.iloc[4:]
vehicle_fuel_df.head()

# renumber index
vehicle_fuel_df = vehicle_fuel_df.reset_index(drop=True)

# melt the dataframe
vehicle_fuel_df = pd.melt(vehicle_fuel_df, id_vars=['Units', 'BodyType', 'Fuel [note 2]', 'Keepership [note 3]', 'ONS Sort [note 6]', 
                                                    'ONS Code [note 6]', 'ONS Geography [note 6]'], var_name='Year', 
                                                    value_vars = ['2022 Q4', '2022 Q3', '2022 Q2', '2022 Q1', '2021 Q4', '2021 Q3', 
                                                                  '2021 Q2', '2021 Q1', '2020 Q4', '2020 Q3', '2020 Q2', '2020 Q1', 
                                                                  '2019 Q4', '2019 Q3', '2019 Q2', '2019 Q1', '2018 Q4', '2018 Q3', 
                                                                  '2018 Q2', '2018 Q1', '2017 Q4', '2017 Q3', '2017 Q2', '2017 Q1', 
                                                                  '2016 Q4', '2016 Q3', '2016 Q2', '2016 Q1', '2015 Q4', '2015 Q3', 
                                                                  '2015 Q2', '2015 Q1', '2014 Q4', '2014 Q3', '2014 Q2', '2014 Q1', 
                                                                  '2013 Q4', '2013 Q3', '2013 Q2', '2013 Q1', '2012 Q4', '2012 Q3', 
                                                                  '2012 Q2', '2012 Q1', '2011 Q4', '2011 Q3', '2011 Q2', '2011 Q1', 
                                                                  '2010 Q4', '2010 Q3', '2010 Q2', '2010 Q1', '2009 Q4'])
# rename the column value to number of vehicles
vehicle_fuel_df = vehicle_fuel_df.rename(columns={'value': 'Number of vehicles'})
# rename the columns to remove the square brackets
vehicle_fuel_df = vehicle_fuel_df.rename(columns={'Fuel [note 2]': 'Fuel', 'Keepership [note 3]': 'Keepership',
                                                    'ONS Sort [note 6]': 'ONS Sort', 'ONS Code [note 6]': 'ONS Code',
                                                    'ONS Geography [note 6]': 'ONS Geography'})
vehicle_fuel_df.head()

# COMBINED WITH ELECTRIC VEHICLE DATA

,Units,BodyType,Fuel,Keepership,ONS Sort,ONS Code,ONS Geography,Year,Number of vehicles
0,Thousands,Buses and coaches,Diesel,Company,1.0,K02000001,United Kingdom,2022 Q4,110.929
1,Thousands,Buses and coaches,Diesel,Company,2.0,K03000001,Great Britain,2022 Q4,106.768
2,Thousands,Buses and coaches,Diesel,Company,3.0,E92000001,England,2022 Q4,90.01
3,Thousands,Buses and coaches,Diesel,Company,4.0,E12000001,North East,2022 Q4,4.103
4,Thousands,Buses and coaches,Diesel,Company,5.0,E06000047,County Durham,2022 Q4,0.608


In [139]:
path = "data/veh0142.ods"

# load a sheet based on its index (1 based)
sheet_idx = 4
vehicle_electricity_df = read_ods(path, sheet_idx)

# make third row the header
vehicle_electricity_df.columns = vehicle_electricity_df.iloc[3]
# drop first 4 rows
vehicle_electricity_df = vehicle_electricity_df.iloc[4:]
vehicle_electricity_df.head()

# renumber index
vehicle_electricity_df = vehicle_electricity_df.reset_index(drop=True)
# melt the dataframe
vehicle_electricity_df = pd.melt(vehicle_electricity_df, id_vars=['Units', 'BodyType', 'Fuel', 'Keepership [note 3]', 
                                                                  'ONS Sort [note 6]', 'ONS Code [note 6]', 'ONS Geography [note 6]'], 
                                                                  var_name='Year', 
                                                                  value_vars = ['2022 Q3', '2022 Q2', '2022 Q1', '2021 Q4', '2021 Q3', 
                                                                                '2021 Q2', '2021 Q1', '2020 Q4', '2020 Q3', '2020 Q2',
                                                                                '2020 Q1', '2019 Q4', '2019 Q3', '2019 Q2', '2019 Q1', 
                                                                                '2018 Q4', '2018 Q3', '2018 Q2', '2018 Q1', '2017 Q4', 
                                                                                '2017 Q3', '2017 Q2', '2017 Q1', '2016 Q4', '2016 Q3', 
                                                                                '2016 Q2', '2016 Q1', '2015 Q4', '2015 Q3', '2015 Q2', 
                                                                                '2015 Q1', '2014 Q4', '2014 Q3', '2014 Q2', '2014 Q1', 
                                                                                '2013 Q4', '2013 Q3', '2013 Q2', '2013 Q1', '2012 Q4', 
                                                                                '2012 Q3', '2012 Q2', '2012 Q1', '2011 Q4', '2011 Q3', 
                                                                                '2011 Q2', '2011 Q1', '2010 Q4', '2010 Q3', '2010 Q2', 
                                                                                '2010 Q1', '2009 Q4'])
# rename the column value to "Number of vehicles"
vehicle_electricity_df = vehicle_electricity_df.rename(columns={'value': 'Number of vehicles'})
# rename the columns to remove the square brackets
vehicle_electricity_df = vehicle_electricity_df.rename(columns={'Keepership [note 3]': 'Keepership',
                                                                'ONS Sort [note 6]': 'ONS Sort',
                                                                'ONS Code [note 6]': 'ONS Code',
                                                                'ONS Geography [note 6]': 'ONS Geography'})
vehicle_electricity_df.head()

# COMBINED WITH OTHER VEHICLE DATA

,Units,BodyType,Fuel,Keepership,ONS Sort,ONS Code,ONS Geography,Year,Number of vehicles
0,Number,Buses and coaches,Battery electric,Company,1.0,K02000001,United Kingdom,2022 Q3,1808.0
1,Number,Buses and coaches,Battery electric,Company,2.0,K03000001,Great Britain,2022 Q3,1702.0
2,Number,Buses and coaches,Battery electric,Company,3.0,E92000001,England,2022 Q3,1343.0
3,Number,Buses and coaches,Battery electric,Company,4.0,E12000001,North East,2022 Q3,21.0
4,Number,Buses and coaches,Battery electric,Company,5.0,E06000047,County Durham,2022 Q3,[c]


In [140]:
# combine the two vehicle dataframes
vehicle_df = pd.concat([vehicle_fuel_df, vehicle_electricity_df])

# keep vehicle_df as the only data frame
vehicle_df = vehicle_electricity_df

# replace any text in number of vehicles column with 0
vehicle_df['Number of vehicles'] = vehicle_df['Number of vehicles'].replace('[x]', 0)
vehicle_df['Number of vehicles'] = vehicle_df['Number of vehicles'].replace('[c]', 0)
# if number of vehicles is NaN or not a number, set to 0
vehicle_df['Number of vehicles'] = vehicle_df['Number of vehicles'].fillna(0)
# convert number of vehicles column to integer
vehicle_df['Number of vehicles'] = vehicle_df['Number of vehicles'].astype(int)
# aggregate the number of vehicles by year
vehicle_df = vehicle_df.groupby(['Year', 'Units', 'BodyType', 'Fuel', 'Keepership', 'ONS Sort', 'ONS Code', 'ONS Geography']).agg({'Number of vehicles': 'sum'}).reset_index()
# split the year column into year and quarter
vehicle_df[['Year', 'Quarter']] = vehicle_df['Year'].str.split(' ', expand=True)
# convert year column to integer
vehicle_df['Year'] = vehicle_df['Year'].astype(int)
# convert quarter column to integer
vehicle_df['Quarter'] = vehicle_df['Quarter'].str.replace('Q', '').astype(int)

vehicle_df

# COMBINED DATAFRAME (VEHICLE AND EV)

,Year,Units,BodyType,Fuel,Keepership,ONS Sort,ONS Code,ONS Geography,Number of vehicles,Quarter
0,2009,Number,Buses and coaches,Battery electric,Company,1.0,K02000001,United Kingdom,0,4
1,2009,Number,Buses and coaches,Battery electric,Company,2.0,K03000001,Great Britain,38,4
2,2009,Number,Buses and coaches,Battery electric,Company,3.0,E92000001,England,36,4
3,2009,Number,Buses and coaches,Battery electric,Company,4.0,E12000001,North East,0,4
4,2009,Number,Buses and coaches,Battery electric,Company,5.0,E06000047,County Durham,0,4
...,...,...,...,...,...,...,...,...,...,...
1719063,2022,Number,Total,Total,Total,460.0,N09000010,"Newry, Mourne and Down",693,3
1719064,2022,Number,Total,Total,Total,461.0,[z],Local Authority unknown within Northern Ire...,761,3
1719065,2022,Number,Total,Total,Total,462.0,[z],"Vehicle under disposal, previously GB",15301,3
1719066,2022,Number,Total,Total,Total,463.0,[z],"Vehicle under disposal, previously NI",191,3


In [141]:
# get unique values in Fuel column
vehicle_df.columns

Index(['Year', 'Units', 'BodyType', 'Fuel', 'Keepership', 'ONS Sort',
       'ONS Code', 'ONS Geography', 'Number of vehicles', 'Quarter'],
      dtype='object')

In [142]:
# load a sheet with car models bought and quarter they were purchased
car_models_df = pd.read_csv('data/vehicle-licensing-statistics-2022-data-files/df_VEH0120_GB.csv')
# melt the dataframe
car_models_df = pd.melt(car_models_df, id_vars=['BodyType', 'Make', 'GenModel', 'Model', 'Fuel'], var_name='Year', 
                        value_vars = ['2022Q3', '2022Q2', '2022Q1', '2021Q4', '2021Q3', '2021Q2', '2021Q1', '2020Q4', '2020Q3',
                                        '2020Q2', '2020Q1', '2019Q4', '2019Q3', '2019Q2', '2019Q1', '2018Q4', '2018Q3', '2018Q2',
                                        '2018Q1', '2017Q4', '2017Q3', '2017Q2', '2017Q1', '2016Q4', '2016Q3', '2016Q2', '2016Q1',
                                        '2015Q4', '2015Q3', '2015Q2', '2015Q1', '2014Q4', '2014Q3', '2014Q2', '2014Q1', '2013Q4',
                                        '2013Q3', '2013Q2', '2013Q1', '2012Q4', '2012Q3', '2012Q2', '2012Q1', '2011Q4', '2011Q3',
                                        '2011Q2', '2011Q1', '2010Q4', '2010Q3', '2010Q2', '2010Q1', '2009Q4', '2009Q3', '2009Q2',
                                        '2009Q1', '2008Q4', '2008Q3', '2007Q4', '2006Q4', '2005Q4', '2004Q4', '2003Q4', '2002Q4', 
                                        '2001Q4', '2000Q4', '1999Q4', '1998Q4', '1997Q4', '1996Q4', '1995Q4', '1994Q4'])
# aggregate the number of cars bought by year, make, genmodel, model, fuel and year
# fill the NaN values with 0
car_models_df['value'] = car_models_df['value'].fillna(0)
car_models_df = car_models_df.groupby(['Year', 'BodyType', 'Make', 'GenModel', 'Model', 'Fuel']).agg({'value': 'sum'}).reset_index()
# rename the column value to "Number of vehicles"
car_models_df = car_models_df.rename(columns={'value': 'Number of vehicles'})
car_models_df.head()

,Year,BodyType,Make,GenModel,Model,Fuel,Number of vehicles
0,1994Q4,Buses and coaches,AIXAM,AIXAM MODEL MISSING,MISSING,Diesel,0
1,1994Q4,Buses and coaches,ALBION,ALBION MODEL MISSING,MISSING,Diesel,26
2,1994Q4,Buses and coaches,ALBION,ALBION MODEL MISSING,MISSING,Petrol,7
3,1994Q4,Buses and coaches,ALEXANDER DENNIS,ALEXANDER DENNIS MODEL MISSING,MISSING,Battery electric,0
4,1994Q4,Buses and coaches,ALEXANDER DENNIS,ALEXANDER DENNIS MODEL MISSING,MISSING,Diesel,0


In [143]:
# get data with fuel prices
fuel_prices_df = pd.read_excel('data/Weekly_Fuel_Prices_120623.xlsx', sheet_name='All years', skiprows=7)
# keep columns 1, 2, 7
fuel_prices_df = fuel_prices_df.iloc[:, [0, 1, 6]]
# aggregate by quarter and get mean
fuel_prices_df = fuel_prices_df.groupby(fuel_prices_df['Date'].dt.to_period("Q")).mean()
# aggregate by year and get mean
fuel_prices_df = fuel_prices_df.groupby(fuel_prices_df.index.year).mean()
# reset index
fuel_prices_df = fuel_prices_df.reset_index()
# rename column Date to Year
fuel_prices_df = fuel_prices_df.rename(columns={'Date': 'Year'})
fuel_prices_df

# COMBINED WITH ELECTRICITY PRICES DATA

,Year,ULSP: Pump price (p/litre),ULSD: Pump price (p/litre)
0,2003,75.330148,77.139578
1,2004,80.254930,81.894003
2,2005,86.796860,90.820266
3,2006,91.395871,95.179094
4,2007,94.384374,96.984666
5,2008,107.001178,117.556702
6,2009,99.593804,104.146614
7,2010,116.904146,119.234615
8,2011,133.412774,138.805382
9,2012,135.761462,142.171851


In [144]:
# combine electric and fuel prices dataframes by year and left join fuel to electric
fuel_electric_prices_df = pd.merge(electricity_df, fuel_prices_df, on='Year', how='left')
fuel_electric_prices_df

# combine North Scotland and South Scotland into Scotland and rename
fuel_electric_prices_df.loc[fuel_electric_prices_df['PES area'].str.contains('North Scotland', case=False, na=False), 'PES area'] = 'Scotland'
fuel_electric_prices_df.loc[fuel_electric_prices_df['PES area'].str.contains('South Scotland', case=False, na=False), 'PES area'] = 'Scotland'
# combine Merseyside & North Wales and South Wales into Wales and rename
fuel_electric_prices_df.loc[fuel_electric_prices_df['PES area'].str.contains('Merseyside & North Wales', case=False, na=False), 'PES area'] = 'Wales'
fuel_electric_prices_df.loc[fuel_electric_prices_df['PES area'].str.contains('South Wales', case=False, na=False), 'PES area'] = 'Wales'
# replace Eastern with East of England
fuel_electric_prices_df['PES area'] = fuel_electric_prices_df['PES area'].replace('Eastern', 'East of England')
# replace Yorkshire with Yorkshire and The Humber
fuel_electric_prices_df['PES area'] = fuel_electric_prices_df['PES area'].replace('Yorkshire', 'Yorkshire and The Humber')
# drop rows with PES area as "United Kingdom" and "Southern"
fuel_electric_prices_df = fuel_electric_prices_df[fuel_electric_prices_df['PES area'] != 'United Kingdom']
fuel_electric_prices_df = fuel_electric_prices_df[fuel_electric_prices_df['PES area'] != 'Southern']

# aggregate all the data by year and PES area and get the mean
fuel_electric_prices_df = fuel_electric_prices_df.groupby(['Year', 'PES area']).mean().reset_index()

# show unique values in PES Area column
fuel_electric_prices_df['PES area'].unique()
# COMBINED DATAFRAME (FUEL + ELECTRICITY PRICES)

array(['East Midlands', 'East of England', 'London', 'North East',
       'North West', 'Northern Ireland', 'Scotland', 'South East',
       'South West', 'Wales', 'West Midlands', 'Yorkshire and The Humber'],
      dtype=object)

In [145]:
# set Year to integer
fuel_electric_prices_df['Year'] = fuel_electric_prices_df['Year'].astype(int)
gdhi_population_df['Year'] = gdhi_population_df['Year'].astype(int)
# add fuel_electric_prices_df to gdhi_population_df
gdhi_population_fuel_electric_df = gdhi_population_df.merge(fuel_electric_prices_df, how='inner', left_on=['Year', 'ITL level 1'], right_on=['Year', 'PES area'])
gdhi_population_fuel_electric_df
# drop PES area column

# COMBINED DATAFRAME (GDHI + POPULATION + FUEL + ELECTRICITY PRICES)

,ITL level 3,ITL level,ITL code,Year,gdhi,sex,age,population,ITL level 1,ITL level 2,PES area,Average variable unit price (£/kWh),ULSP: Pump price (p/litre),ULSD: Pump price (p/litre)
0,Aberdeen City and Aberdeenshire,ITL3,TLM50,2010,18736.0,1,0,2801,Scotland,North Eastern Scotland,Scotland,0.117261,116.904146,119.234615
1,Aberdeen City and Aberdeenshire,ITL3,TLM50,2010,18736.0,1,1,2762,Scotland,North Eastern Scotland,Scotland,0.117261,116.904146,119.234615
2,Aberdeen City and Aberdeenshire,ITL3,TLM50,2010,18736.0,1,2,2741,Scotland,North Eastern Scotland,Scotland,0.117261,116.904146,119.234615
3,Aberdeen City and Aberdeenshire,ITL3,TLM50,2010,18736.0,1,3,2596,Scotland,North Eastern Scotland,Scotland,0.117261,116.904146,119.234615
4,Aberdeen City and Aberdeenshire,ITL3,TLM50,2010,18736.0,1,4,2501,Scotland,North Eastern Scotland,Scotland,0.117261,116.904146,119.234615
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
354349,West Northamptonshire,ITL3,TLF24,2020,22354.0,2,86,768,East Midlands,"Leicestershire, Rutland and Northamptonshire",East Midlands,0.166876,114.092268,119.455858
354350,West Northamptonshire,ITL3,TLF24,2020,22354.0,2,87,621,East Midlands,"Leicestershire, Rutland and Northamptonshire",East Midlands,0.166876,114.092268,119.455858
354351,West Northamptonshire,ITL3,TLF24,2020,22354.0,2,88,586,East Midlands,"Leicestershire, Rutland and Northamptonshire",East Midlands,0.166876,114.092268,119.455858
354352,West Northamptonshire,ITL3,TLF24,2020,22354.0,2,89,451,East Midlands,"Leicestershire, Rutland and Northamptonshire",East Midlands,0.166876,114.092268,119.455858


In [146]:

# remove all rows with ONS Geography 'Vehicle under disposal, previously GB','Vehicle under disposal, previously NI','Region or county unknown'
vehicle_removed_df = vehicle_df[~vehicle_df['ONS Geography'].isin(['Vehicle under disposal, previously NI','Region or county unknown'])]
# strip whitespace from ONS Geography column from front and back
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].str.strip()

vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(['Barking and Dagenham', 'Havering'], 'Barking & Dagenham and Havering')
# change "Angus" to "Angus and Dundee City"
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(['Angus', 'Dundee City'], 'Angus and Dundee City')
# change Maldon to Heart of Essex
vehicle_removed_df.loc[vehicle_removed_df['ONS Geography'].str.contains('Maldon', case=False, na=False), 'ONS Geography'] = 'Heart of Essex'
# change the values in laname21 column to match the values in gdhi data "Region name" columns
# change "Hartlepool" to "Durham CC"
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(['Hartlepool'], 'Hartlepool and Stockton-on-Tees')
# change "Aberdeen City" to "Aberdeen City and Aberdeenshire"
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(['Aberdeen City'], 'Aberdeen City and Aberdeenshire')
# change "Aberdeenshire" to "Aberdeen City and Aberdeenshire"
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(['Aberdeenshire'], 'Aberdeen City and Aberdeenshire')
# change "Adur" to "West Sussex (South West)"
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(['Worthing', 'Adur', 'Arun', 'Chichester', 'Horsham', 'Mid Sussex'], 'West Sussex (South West)')
# change "Allerdale" to "West Cumbria"
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(['Allerdale', 'Barrow-in-Furness', 'Carlisle', 'Copeland'], 'West Cumbria')
# change "Allerdale" to "West Cumbria"
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(['Eden', 'South Lakeland'], 'East Cumbria')
# change "Argyll and Bute" to "Lochaber, Skye and Lochalsh, Arran and Cumbrae and Argyll and Bute"
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(['Argyll and Bute'], 'Lochaber, Skye and Lochalsh, Arran and Cumbrae and Argyll and Bute')
# change Ashfield to "North Nottinghamshire"
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(['Ashfield', 'Bassetlaw', 'Bolsover', 'Gedling'], 'North Nottinghamshire')
# change Babergh to "Suffolk"
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(['Babergh', 'East Suffolk', 'Ipswich', 'Mid Suffolk', 'West Suffolk'], 'Suffolk')
# change Barnsley to "Barnsley, Doncaster and Rotherham"
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(['Barnsley', 'Doncaster', 'Rotherham'], 'Barnsley, Doncaster and Rotherham')
# change Basildon to "Thurrock"
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(['Basildon'], 'Thurrock')
# change Basingstoke and Deane to "North Hampshire"
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(['Basingstoke and Deane', 'Hart', 'Rushmoor'], 'North Hampshire')
# change Bath and North East Somerset to "Bath and North East Somerset, North Somerset and South Gloucestershire"
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(['Bath and North East Somerset', 'North Somerset', 'South Gloucestershire'], 'Bath and North East Somerset, North Somerset and South Gloucestershire')
# change Bexley to "Bexley and Greenwich"
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(['Bexley', 'Greenwich'], 'Bexley and Greenwich')
# change Blaby to "Leicestershire CC and Rutland"
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(['Blaby', 'Charnwood', 'Harborough', 'Hinckley and Bosworth', 'Melton', 'North West Leicestershire', 'Oadby and Wigston', 'Rutland'], 'Leicestershire CC and Rutland')
# change Blaenau Gwent to "Gwent Valleys"
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(['Blaenau Gwent', 'Torfaen'], 'Gwent Valleys')
# change Bolton to "Greater Manchester North West"
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(['Bolton', 'Bury'], 'Greater Manchester North West')
# change Boston to "Lincolnshire"
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(['Boston', 'North Kesteven', 'West Lindsey', 'South Kesteven', 'South Holland'], 'Lincolnshire')
# change Bracknell Forest to "Berkshire"
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(['Bracknell Forest', 'Reading', 'Slough', 'West Berkshire', 'Windsor and Maidenhead', 'Wokingham'], 'Berkshire')
# change Braintree to "Essex Haven Gateway"
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(['Braintree', 'Brentwood', 'Castle Point', 'Colchester', 'Chelmsford', 'Rochford'], 'Essex Haven Gateway')
# change Breckland to "Breckland and South Norfolk"
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(['Breckland', 'Broadland', 'Great Yarmouth', 'South Norfolk'], 'Breckland and South Norfolk')
# change Bridgend to "Bridgend and Neath Port Talbot"
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(['Bridgend', 'Merthyr Tydfil', 'Neath Port Talbot', 'Rhondda Cynon Taf'], 'Bridgend and Neath Port Talbot')
#  change Bromsgrove to "Worcestershire"
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(['Bromsgrove', 'Malvern Hills', 'Redditch', 'Worcester', 'Wychavon'], 'Worcestershire')
# change Broxbourne to "Hertfordshire"
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(['Watford', 'Stevenage', 'St Albans', 'Three Rivers', 'Welwyn Hatfield', 'North Hertfordshire', 'Broxbourne', 'East Hertfordshire', 'Dacorum', 'Hertsmere'], 'Hertfordshire')
# change Broxtowe to "South and West Derbyshire"
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(['Broxtowe', 'Amber Valley', 'Derbyshire Dales', 'Erewash', 'South Derbyshire'], 'South and West Derbyshire')
# change Buckinghamshire to "Buckinghamshire CC"
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(['Buckinghamshire'], 'Buckinghamshire CC')
# change Burnley to "East Lancashire"
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(['Burnley', 'Hyndburn', 'Pendle', 'Ribble Valley', 'Rossendale'], 'East Lancashire')
# change Caerphilly to "Cardiff and Vale of Glamorgan"
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(['Caerphilly', 'Cardiff', 'Vale of Glamorgan'], 'Cardiff and Vale of Glamorgan')
# change Cambridge to Cambridgeshire CC
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(['Cambridge', 'East Cambridgeshire', 'Fenland', 'Huntingdonshire', 'South Cambridgeshire'], 'Cambridgeshire CC')
# change Cannock Chase to Staffordshire CC
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(['Newcastle-under-Lyme', 'Cannock Chase', 'East Staffordshire', 'Lichfield', 'South Staffordshire', 'Tamworth', 'Staffordshire Moorlands', 'Stafford'], 'Staffordshire CC')
# change Carmarthenshire to South West Wales
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(['Carmarthenshire', 'Pembrokeshire'], 'South West Wales')
# change Ceredigion to Central Valleys
vehicle_removed_df.loc[vehicle_removed_df['ONS Geography'].str.contains('Ceredigion', case=False, na=False), 'ONS Geography'] = 'Central Valleys'
# change Cherwell to Oxfordshire
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(['Cherwell', 'Oxford', 'South Oxfordshire', 'Vale of White Horse', 'West Oxfordshire'], 'Oxfordshire')
# change Chesterfield to East Derbyshire
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(['Chesterfield', 'North East Derbyshire', 'High Peak'], 'East Derbyshire')
# change Chorley to Chorley and West Lancashire
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(['Chorley', 'Fylde', 'South Ribble', 'West Lancashire'], 'Chorley and West Lancashire')
# change City of London to Camden and City of London
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(['City of London', 'Camden'], 'Camden and City of London')
# change Clackmannanshire to Clackmannanshire and Fife
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(['Clackmannanshire', 'Fife'], 'Clackmannanshire and Fife')
# change Conwy to Conwy and Denbighshire
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(['Conwy', 'Denbighshire'], 'Conwy and Denbighshire')
# change Cornwall to Cornwall and Isles of Scilly
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(['Cornwall', 'Isles of Scilly'], 'Cornwall and Isles of Scilly')
# change County Durham to Durham CC
vehicle_removed_df.loc[vehicle_removed_df['ONS Geography'].str.contains('County Durham', case=False, na=False), 'ONS Geography'] = 'Durham CC'
# change Crawley to West Sussex
vehicle_removed_df.loc[vehicle_removed_df['ONS Geography'].str.contains('Crawley', case=False, na=False), 'ONS Geography'] = 'West Sussex (North East)'
# change East Ayrshire to East Ayrshire and North Ayrshire mainland
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(['East Ayrshire', 'North Ayrshire'], 'East Ayrshire and North Ayrshire mainland')
# change East Devon to Devon CC
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(['East Devon', 'Exeter', 'Mid Devon', 'North Devon', 'South Hams', 'Teignbridge', 'Torridge', 'West Devon'], 'Devon CC')
# change East Dunbartonshire to East Dunbartonshire, West Dunbartonshire and Helensburgh and Lomond
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(['East Dunbartonshire', 'West dunbartonshire', 'West Dunbartonshire'], 'East Dunbartonshire, West Dunbartonshire and Helensburgh and Lomond')
# change East Hampshire to Central Hampshire
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(['East Hampshire', 'Eastleigh', 'New Forest'], 'Central Hampshire')
# change East Lothian to East Lothian and Midlothian
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(['East Lothian', 'Midlothian'], 'East Lothian and Midlothian')
# change East Renfrewshire to Inverclyde, East Renfrewshire and Renfrewshire
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(['East Renfrewshire', 'Inverclyde', 'Renfrewshire'], 'Inverclyde, East Renfrewshire and Renfrewshire')
# change Eastbourne to East Sussex
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(['Eastbourne', 'East Sussex'])
# change Elmbridge to West Surrey
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(['Elmbridge', 'Epsom and Ewell', 'Guildford', 'Runnymede', 'Spelthorne', 'Woking', 'Waverley', 'Surrey Heath'], 'West Surrey')
# change Fareham to South Hampshire
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(['Fareham', 'Gosport', 'Havant'], 'South Hampshire')
# change Flintshire to Flintshire and Wrexham
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(['Flintshire', 'Wrexham'], 'Flintshire and Wrexham')
# change Folkestone and Hythe to East Kent
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(['Folkestone and Hythe', 'Dover', 'Ashford', 'Canterbury'], 'East Kent')
# change Forest of Dean to Gloucestershire
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(['Gloucester', 'Forest of Dean', 'Cotswold', 'Cheltenham', 'Stroud', 'Tewkesbury'], 'Gloucestershire')
# change Gateshead to Tyneside
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(['Newcastle upon Tyne', 'Gateshead', 'North Tyneside', 'South Tyneside'], 'Tyneside')
# change Hackney to Hackney and Newham
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(['Hackney', 'Newham'], 'Hackney and Newham')
# change Hammersmith and Fulham to Kensington & Chelsea and Hammersmith & Fulham
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(['Hammersmith and Fulham', 'Kensington and Chelsea'], 'Kensington & Chelsea and Hammersmith & Fulham')
# change Harlow to Essex Thames Gateway
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(['Harlow', 'Epping Forest'], 'Essex Thames Gateway')
# change Harrow to Harrow and Hillingdon
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(['Harrow', 'Hillingdon'], 'Harrow and Hillingdon')
# change Haringey to Haringey and Islington
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(['Haringey', 'Islington'], 'Haringey and Islington')
# change Hastings to East Sussex CC
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(['Hastings', 'Lewes', 'Rother', 'Wealden'], 'East Sussex CC')
# change Highland to Highlands and Islands
vehicle_removed_df.loc[vehicle_removed_df['ONS Geography'].str.contains('Highland', case=False, na=False), 'ONS Geography'] = 'Caithness and Sutherland and Ross and Cromarty'
# change Hounslow to Hounslow and Richmond upon Thames
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(['Hounslow', 'Richmond upon Thames'], 'Hounslow and Richmond upon Thames')
# change King's Lynn and West Norfolk to North and West Norfolk
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(["King's Lynn and West Norfolk", 'North Norfolk'], 'North and West Norfolk')
# change Kingston upon Thames to Merton, Kingston upon Thames and Sutton
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(['Kingston upon Thames', 'Merton', 'Sutton'], 'Merton, Kingston upon Thames and Sutton')
# change Kirklees to Calderdale and Kirklees
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(['Kirklees', 'Calderdale'], 'Calderdale and Kirklees')
# change Knowsley to East Merseyside
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(['Knowsley', 'St. Helens'], 'East Merseyside')
# change Lancaster to Lancaster and Wyre
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(['Lancaster', 'Wyre', 'Wyre Forest'], 'Lancaster and Wyre')
# change Lewisham to Lewisham and Southwark
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(['Lewisham', 'Southwark'], 'Lewisham and Southwark')
# change Lincoln to North and North East Lincolnshire
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(['Lincoln', 'North East Lincolnshire', 'North Lincolnshire', 'East Lindsey'], 'North and North East Lincolnshire')
# change Maidstone to Medway
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace('Maidstone', 'Medway')
# change Mansfield to Nottingham
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(['Mansfield', 'Newark and Sherwood'], 'Nottingham')
# change Mendip to Somerset
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(['Mendip', 'South Somerset', 'Somerset West and Taunton', 'Sedgemoor'], 'Somerset')
# change Mole Valley to East Surrey
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(['Mole Valley', 'Reigate and Banstead', 'Tandridge'], 'East Surrey')
# change Monmouthshire to Monmouthshire and Newport
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(['Monmouthshire', 'Newport'], 'Monmouthshire and Newport')
# change Moray to Inverness and Nairn and Moray, Badenoch and Strathspey
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(['Moray'], 'Inverness and Nairn and Moray, Badenoch and Strathspey')
# change North Warwickshire to Warwickshire
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(['Warwick', 'North Warwickshire', 'Nuneaton and Bedworth', 'Rugby', 'Stratford-on-Avon'], 'Warwickshire')
# change Norwich to Norwich and East Norfolk
vehicle_removed_df.loc[vehicle_removed_df['ONS Geography'].str.contains('Norwich', case=False, na=False), 'ONS Geography'] = 'Norwich and East Norfolk'
# change Oldham to Greater Manchester North East
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(['Oldham', 'Rochdale', 'Salford'], 'Greater Manchester North East')
# change Perth and Kinross to Perth and Kinross and Stirling
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(['Perth and Kinross', 'Stirling'], 'Perth and Kinross and Stirling')
# change preston to Mid Lancashire
vehicle_removed_df.loc[vehicle_removed_df['ONS Geography'].str.contains('Preston', case=False, na=False), 'ONS Geography'] = 'Mid Lancashire'
# change 'Redcar and Cleveland' to South Teesside
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(['Redcar and Cleveland'], 'South Teesside')
# change Redcar and Cleveland to North Yorkshire CC
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(['Richmondshire', 'Middlesbrough', 'Hambleton', 'Harrogate', 'Ryedale', 'Scarborough', 'Selby', 'Craven'], 'North Yorkshire CC')
# change Redbridge to Redbridge and Waltham Forest
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(['Redbridge', 'Waltham Forest'], 'Redbridge and Waltham Forest')
# change Rushcliffe to Nottinghamshire
vehicle_removed_df.loc[vehicle_removed_df['ONS Geography'].str.contains('Rushcliffe', case=False, na=False), 'ONS Geography'] = 'South Nottinghamshire'
# change Shropshire to Shropshire CC
vehicle_removed_df.loc[vehicle_removed_df['ONS Geography'].str.contains('Shropshire', case=False, na=False), 'ONS Geography'] = 'Shropshire CC'
# change Stockport to Greater Manchester South East
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(['Stockport', 'Tameside', 'Wigan'], 'Greater Manchester South East')
# change 'Trafford' to Greater Manchester South West
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(['Trafford'], 'Greater Manchester South West')
# change Stockton-on-Tees to Hartlepool and Stockton-on-Tees
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(['Stockton-on-Tees', 'Hartlepool and Stockton-on-Tees'])
# change Tendring to Essex
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(['Tendring', 'Uttlesford'], 'West Essex')
# change Test Valley to Central Hampshire
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(['Test Valley'], 'Central Hampshire')
# change Winchester to Portsmouth
vehicle_removed_df.loc[vehicle_removed_df['ONS Geography'].str.contains('Winchester', case=False, na=False), 'ONS Geography'] = 'Portsmouth'
# 'Tonbridge and Malling' to west Kent
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(['Tonbridge and Malling', 'Tunbridge Wells', 'Sevenoaks'], 'West Kent')
# change Tunbridge Wells to Kent Thames Gateway
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(['Thanet', 'Swale', 'Gravesham', 'Dartford'], 'Kent Thames Gateway')




# check which values are in ONS Geography in vehicle_removed_df but not in gdhi_population_fuel_electric_df
remove = set(vehicle_removed_df['ONS Geography'].unique()) - set(gdhi_population_fuel_electric_df['ITL level 3'].unique())
# remove rows with values in remove
vehicle_removed_df = vehicle_removed_df[~vehicle_removed_df['ONS Geography'].isin(remove)]

/var/folders/j9/0qd6cw014pgb7rk0v_28prrh0000gn/T/ipykernel_15876/1121752590.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].str.strip()
/var/folders/j9/0qd6cw014pgb7rk0v_28prrh0000gn/T/ipykernel_15876/1121752590.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(['Barking and Dagenham', 'Havering'], 'Barking & Dagenham and Havering')
/var/folders/j9

In [147]:
# get unique values in Fuel column
vehicle_removed_df['Fuel'].unique()
# columns
vehicle_removed_df.columns
# remove ONS Code and ONS Sort columns
vehicle_removed_df = vehicle_removed_df.drop(columns=['ONS Code', 'ONS Sort'])

In [148]:
#Fuel types - ['Battery electric', 'Total', 'Plug-in hybrid electric (diesel)', 'Plug-in hybrid electric (petrol)', 'Range extended electric','Diesel', 'Hybrid electric (petrol)', 'Other fuels', 'Petrol']
# create new dataframe for each fuel type
battery_electric_df = vehicle_removed_df[vehicle_removed_df['Fuel'] == 'Battery electric']
plug_in_hybrid_diesel_df = vehicle_removed_df[vehicle_removed_df['Fuel'] == 'Plug-in hybrid electric (diesel)']
plug_in_hybrid_petrol_df = vehicle_removed_df[vehicle_removed_df['Fuel'] == 'Plug-in hybrid electric (petrol)']
range_extended_electric_df = vehicle_removed_df[vehicle_removed_df['Fuel'] == 'Range extended electric']
diesel_df = vehicle_removed_df[vehicle_removed_df['Fuel'] == 'Diesel']
hybrid_petrol_df = vehicle_removed_df[vehicle_removed_df['Fuel'] == 'Hybrid electric (petrol)']
other_fuels_df = vehicle_removed_df[vehicle_removed_df['Fuel'] == 'Other fuels']
petrol_df = vehicle_removed_df[vehicle_removed_df['Fuel'] == 'Petrol']
total_df = vehicle_removed_df[vehicle_removed_df['Fuel'] == 'Total']


In [149]:
# merge each fuel type dataframe with gdhi_population_fuel_electric_df
#battery_electric_gdhi_df = gdhi_population_fuel_electric_df.merge(battery_electric_df, how='inner', left_on=['Year', 'ITL level 3'], right_on=['Year', 'ONS Geography'])
#plug_in_hybrid_diesel_gdhi_df = gdhi_population_fuel_electric_df.merge(plug_in_hybrid_diesel_df, how='inner', left_on=['Year', 'ITL level 3'], right_on=['Year', 'ONS Geography'])
#plug_in_hybrid_petrol_gdhi_df = gdhi_population_fuel_electric_df.merge(plug_in_hybrid_petrol_df, how='inner', left_on=['Year', 'ITL level 3'], right_on=['Year', 'ONS Geography'])
#range_extended_electric_gdhi_df = gdhi_population_fuel_electric_df.merge(range_extended_electric_df, how='inner', left_on=['Year', 'ITL level 3'], right_on=['Year', 'ONS Geography'])
#diesel_gdhi_df = gdhi_population_fuel_electric_df.merge(diesel_df, how='inner', left_on=['Year', 'ITL level 3'], right_on=['Year', 'ONS Geography'])
#hybrid_petrol_gdhi_df = gdhi_population_fuel_electric_df.merge(hybrid_petrol_df, how='inner', left_on=['Year', 'ITL level 3'], right_on=['Year', 'ONS Geography'])
#other_fuels_gdhi_df = gdhi_population_fuel_electric_df.merge(other_fuels_df, how='inner', left_on=['Year', 'ITL level 3'], right_on=['Year', 'ONS Geography'])
#petrol_gdhi_df = gdhi_population_fuel_electric_df.merge(petrol_df, how='inner', left_on=['Year', 'ITL level 3'], right_on=['Year', 'ONS Geography'])
total_gdhi_df = gdhi_population_fuel_electric_df.merge(total_df, how='inner', left_on=['Year', 'ITL level 3'], right_on=['Year', 'ONS Geography'])

# drop ONS Geography column
#battery_electric_gdhi_df = battery_electric_gdhi_df.drop(columns=['ONS Geography'])
#plug_in_hybrid_diesel_gdhi_df = plug_in_hybrid_diesel_gdhi_df.drop(columns=['ONS Geography'])
#plug_in_hybrid_petrol_gdhi_df = plug_in_hybrid_petrol_gdhi_df.drop(columns=['ONS Geography'])
#range_extended_electric_gdhi_df = range_extended_electric_gdhi_df.drop(columns=['ONS Geography'])
#diesel_gdhi_df = diesel_gdhi_df.drop(columns=['ONS Geography'])
#hybrid_petrol_gdhi_df = hybrid_petrol_gdhi_df.drop(columns=['ONS Geography'])
#other_fuels_gdhi_df = other_fuels_gdhi_df.drop(columns=['ONS Geography'])
#petrol_gdhi_df = petrol_gdhi_df.drop(columns=['ONS Geography'])
total_gdhi_df = total_gdhi_df.drop(columns=['ONS Geography'])

# drop nan values
#battery_electric_gdhi_df = battery_electric_gdhi_df.dropna()
#plug_in_hybrid_diesel_gdhi_df = plug_in_hybrid_diesel_gdhi_df.dropna()
#plug_in_hybrid_petrol_gdhi_df = plug_in_hybrid_petrol_gdhi_df.dropna()
#range_extended_electric_gdhi_df = range_extended_electric_gdhi_df.dropna()
#diesel_gdhi_df = diesel_gdhi_df.dropna()
#hybrid_petrol_gdhi_df = hybrid_petrol_gdhi_df.dropna()
#other_fuels_gdhi_df = other_fuels_gdhi_df.dropna()
#petrol_gdhi_df = petrol_gdhi_df.dropna()
total_gdhi_df = total_gdhi_df.dropna()




In [150]:
chargepoint_number_df = chargepoint_df
# drop rows with na value for dateCreated
chargepoint_number_df = chargepoint_number_df.dropna(subset=['dateCreated'])
# drop rows with 0000-00-00 00:00:00 value for dateCreated
chargepoint_number_df = chargepoint_number_df[chargepoint_number_df['dateCreated'] != '0000-00-00 00:00:00']
# drop rows with 230 value for dateCreated
chargepoint_number_df = chargepoint_number_df[chargepoint_number_df['dateCreated'] != '230']
# convert dateCreated to datetime
chargepoint_number_df['dateCreated'] = pd.to_datetime(chargepoint_number_df['dateCreated'],format='%Y-%m-%d %H:%M:%S')

# extract year from dateCreated
chargepoint_number_df['Year'] = chargepoint_number_df['dateCreated'].dt.year
# extract quarter from dateCreated
chargepoint_number_df['Quarter'] = chargepoint_number_df['dateCreated'].dt.quarter

# get number of chargepoints per year in each county and show sum of number of rows in each year
chargepoint_number_df = chargepoint_number_df.groupby(['Year', 'county','Quarter']).size().reset_index(name='Number of Chargepoints').sort_values(by=['Year', 'county'])

# rename county column to ITL level 3
chargepoint_number_df = chargepoint_number_df.rename(columns={'county': 'ITL level 3'})

# remove rows with values 0, 4
chargepoint_number_df = chargepoint_number_df[~chargepoint_number_df['ITL level 3'].isin(['0', '4', 'County Cork','County Dublin', 
                                                                                          'Dublin', 'Guernsey', 'Me10 2La', 'Notts.', 'Notts', 
                                                                                          'Oxon', 'SG1 1EP', 'Staffs', 'Warcs', 'Warks', 
                                                                                           'Wick','Wilts', 'Worcs', 'Yorks', '`West Midlands',
                                                                                            'United Kingdom', 'None', 'Lincs', 'Leics', 'Lancs', 
                                                                                             'Gwent','Herts', 'Hants', 'Gloucs', 'Derbys',
                                                                                             'County Galway','Galway County', 'County Limerick',
                                                                                             'West Midland','West Midlands','West Mdilands',
                                                                                            'County Wicklow'])]
# strip whitespace from ITL level 3 column from front and back
chargepoint_number_df['ITL level 3'] = chargepoint_number_df['ITL level 3'].str.strip()




chargepoint_number_df['ITL level 3'] = chargepoint_number_df['ITL level 3'].replace(['Barking and Dagenham', 'Havering'], 'Barking & Dagenham and Havering')
# change "Angus" to "Angus and Dundee City"
chargepoint_number_df['ITL level 3'] = chargepoint_number_df['ITL level 3'].replace(['Angus', 'Dundee City','Dundee'], 'Angus and Dundee City')
# change Maldon to Heart of Essex
chargepoint_number_df.loc[chargepoint_number_df['ITL level 3'].str.contains('Maldon', case=False, na=False), 'ITL level 3'] = 'Heart of Essex'
# change the values in laname21 column to match the values in gdhi data "Region name" columns
# change "Hartlepool" to "Durham CC"
chargepoint_number_df['ITL level 3'] = chargepoint_number_df['ITL level 3'].replace(['Hartlepool'], 'Hartlepool and Stockton-on-Tees')
# change "Aberdeen City" to "Aberdeen City and Aberdeenshire"
chargepoint_number_df['ITL level 3'] = chargepoint_number_df['ITL level 3'].replace(['Aberdeen City'], 'Aberdeen City and Aberdeenshire')
# change "Aberdeenshire" to "Aberdeen City and Aberdeenshire"
chargepoint_number_df['ITL level 3'] = chargepoint_number_df['ITL level 3'].replace(['City of Aberdeen','Aberdeenshire'], 'Aberdeen City and Aberdeenshire')
# change "Adur" to "West Sussex (South West)"
chargepoint_number_df['ITL level 3'] = chargepoint_number_df['ITL level 3'].replace(['Worthing', 'Adur', 'Arun', 'Chichester', 'Horsham', 'Mid Sussex'], 'West Sussex (South West)')
# change "Allerdale" to "West Cumbria"
chargepoint_number_df['ITL level 3'] = chargepoint_number_df['ITL level 3'].replace(['Cumbria','Allerdale', 'Barrow-in-Furness', 'Carlisle', 'Copeland'], 'West Cumbria')
# change "Allerdale" to "West Cumbria"
chargepoint_number_df['ITL level 3'] = chargepoint_number_df['ITL level 3'].replace(['Eden', 'South Lakeland'], 'East Cumbria')
# change "Argyll and Bute" to "Lochaber, Skye and Lochalsh, Arran and Cumbrae and Argyll and Bute"
chargepoint_number_df['ITL level 3'] = chargepoint_number_df['ITL level 3'].replace(['Argyll and Bute', 'Argyl and Bute', 'Argyll', 'Argyll & Bute'], 'Lochaber, Skye and Lochalsh, Arran and Cumbrae and Argyll and Bute')
# change Ashfield to "North Nottinghamshire"
chargepoint_number_df['ITL level 3'] = chargepoint_number_df['ITL level 3'].replace(['Ashfield', 'Bassetlaw', 'Bolsover', 'Gedling'], 'North Nottinghamshire')
# change Babergh to "Suffolk"
chargepoint_number_df['ITL level 3'] = chargepoint_number_df['ITL level 3'].replace(['Babergh', 'East Suffolk', 'Ipswich', 'Mid Suffolk', 'West Suffolk'], 'Suffolk')
# change Barnsley to "Barnsley, Doncaster and Rotherham"
chargepoint_number_df['ITL level 3'] = chargepoint_number_df['ITL level 3'].replace(['Barnsley', 'Doncaster', 'Rotherham'], 'Barnsley, Doncaster and Rotherham')
# change Basildon to "Thurrock"
chargepoint_number_df['ITL level 3'] = chargepoint_number_df['ITL level 3'].replace(['Basildon'], 'Thurrock')
# change Basingstoke and Deane to "North Hampshire"
chargepoint_number_df['ITL level 3'] = chargepoint_number_df['ITL level 3'].replace(['Basingstoke','Basingstoke and Deane Borough Council','Basingstoke and Deane', 'Hart', 'Rushmoor'], 'North Hampshire')
# change Bath and North East Somerset to "Bath and North East Somerset, North Somerset and South Gloucestershire"
chargepoint_number_df['ITL level 3'] = chargepoint_number_df['ITL level 3'].replace(['Bath and North East Somerset', 'North Somerset', 'South Gloucestershire'], 'Bath and North East Somerset, North Somerset and South Gloucestershire')
# change Bexley to "Bexley and Greenwich"
chargepoint_number_df['ITL level 3'] = chargepoint_number_df['ITL level 3'].replace(['Bexley', 'Greenwich'], 'Bexley and Greenwich')
# change Blaby to "Leicestershire CC and Rutland"
chargepoint_number_df['ITL level 3'] = chargepoint_number_df['ITL level 3'].replace(['Blaby', 'Rutlands', 'Charnwood', 'Harborough', 'Hinckley and Bosworth', 'Melton','City of Leicester', 'Leicestershire', 'North West Leicestershire', 'Oadby and Wigston', 'Rutland'], 'Leicestershire CC and Rutland')
# change Blaenau Gwent to "Gwent Valleys"
chargepoint_number_df['ITL level 3'] = chargepoint_number_df['ITL level 3'].replace(['Blaenau Gwent', 'Torfaen'], 'Gwent Valleys')
# change Bolton to "Greater Manchester North West"
chargepoint_number_df['ITL level 3'] = chargepoint_number_df['ITL level 3'].replace(['Bolton', 'Greater Manchester', 'Bury'], 'Greater Manchester North West')
# change Boston to "Lincolnshire"
chargepoint_number_df['ITL level 3'] = chargepoint_number_df['ITL level 3'].replace(['Boston', 'Grantham', 'North Kesteven', 'West Lindsey', 'South Kesteven', 'South Holland'], 'Lincolnshire')
# change Bracknell Forest to "Berkshire"
chargepoint_number_df['ITL level 3'] = chargepoint_number_df['ITL level 3'].replace(['Bracknell Forest', 'Reading', 'Slough', 'West Berkshire', 'Windsor and Maidenhead', 'Wokingham', 'BERKSHIRE', 'West Berkshire Council'], 'Berkshire')
# change Braintree to "Essex Haven Gateway"
chargepoint_number_df['ITL level 3'] = chargepoint_number_df['ITL level 3'].replace(['Braintree', 'Brentwood', 'Castle Point', 'Colchester', 'Chelmsford', 'Rochford'], 'Essex Haven Gateway')
# change Breckland to "Breckland and South Norfolk"
chargepoint_number_df['ITL level 3'] = chargepoint_number_df['ITL level 3'].replace(['Breckland', 'Broadland', 'Great Yarmouth', 'South Norfolk'], 'Breckland and South Norfolk')
# change Bridgend to "Bridgend and Neath Port Talbot"
chargepoint_number_df['ITL level 3'] = chargepoint_number_df['ITL level 3'].replace(['Bridgend', 'Merthyr Tydfil', 'Neath Port Talbot', 'Rhondda Cynon Taf'], 'Bridgend and Neath Port Talbot')
#  change Bromsgrove to "Worcestershire"
chargepoint_number_df['ITL level 3'] = chargepoint_number_df['ITL level 3'].replace(['Bromsgrove', 'Malvern Hills', 'Redditch', 'Worcester', 'Wychavon'], 'Worcestershire')
# change Broxbourne to "Hertfordshire"
chargepoint_number_df['ITL level 3'] = chargepoint_number_df['ITL level 3'].replace(['Watford', 'Stevenage', 'St Albans', 'Three Rivers', 'Welwyn Hatfield', 'North Hertfordshire', 'Broxbourne', 'East Hertfordshire', 'Dacorum', 'Hertsmere' 'Hereford','Herefordshire',], 'Hertfordshire')
# change Broxtowe to "South and West Derbyshire"
chargepoint_number_df['ITL level 3'] = chargepoint_number_df['ITL level 3'].replace(['Broxtowe', 'Amber Valley', 'Derbyshire Dales', 'Erewash', 'South Derbyshire'], 'South and West Derbyshire')
# change Buckinghamshire to "Buckinghamshire CC"
chargepoint_number_df['ITL level 3'] = chargepoint_number_df['ITL level 3'].replace(['City of Westminster', 'Buckinghamshire','Buckingham','Bucks'], 'Buckinghamshire CC')
# change Burnley to "East Lancashire"
chargepoint_number_df['ITL level 3'] = chargepoint_number_df['ITL level 3'].replace(['Burnley', 'Hyndburn', 'Pendle', 'Ribble Valley', 'Rossendale'], 'East Lancashire')
# change Caerphilly to "Cardiff and Vale of Glamorgan"
chargepoint_number_df['ITL level 3'] = chargepoint_number_df['ITL level 3'].replace(['City of Cardiff', 'Glamorgan','Glamorgan (Morgannwg)',
                                                                                     'Glamorganshire','Caerphilly', 'Cardiff', 'Vale of Glamorgan',
                                                                                     'Vale of Glamorgan, The',  'West Glamorgan', 'Mid Glamorgan', 
                                                                                     'South Glamorgan'], 'Cardiff and Vale of Glamorgan')
# change Cambridge to Cambridgeshire CC
chargepoint_number_df['ITL level 3'] = chargepoint_number_df['ITL level 3'].replace(['Cambridge','Cambs', 'East Cambridgeshire', 'Fenland', 'Huntingdonshire', 'South Cambridgeshire','Cambridgeshire'], 'Cambridgeshire CC')
# change Cannock Chase to Staffordshire CC
chargepoint_number_df['ITL level 3'] = chargepoint_number_df['ITL level 3'].replace(['Burton-on-Trent','Newcastle-under-Lyme', 'Cannock Chase', 'East Staffordshire', 'Lichfield', 'South Staffordshire', 'Tamworth', 'Staffordshire Moorlands', 'Stafford'], 'Staffordshire CC')
# change Carmarthenshire to South West Wales
chargepoint_number_df['ITL level 3'] = chargepoint_number_df['ITL level 3'].replace(['Carmarthenshire', 'Pembrokeshire'], 'South West Wales')
# change Aberystwyth to Ceredigion
chargepoint_number_df['ITL level 3'] = chargepoint_number_df['ITL level 3'].replace(['Aberystwyth'], 'Ceredigion')
# change Ceredigion to Central Valleys
chargepoint_number_df.loc[chargepoint_number_df['ITL level 3'].str.contains('Ceredigion', case=False, na=False), 'ITL level 3'] = 'Central Valleys'
# change Cherwell to Oxfordshire
chargepoint_number_df['ITL level 3'] = chargepoint_number_df['ITL level 3'].replace(['Oxford City Council','Cherwell', 'Oxford', 'South Oxfordshire', 'Vale of White Horse', 'West Oxfordshire'], 'Oxfordshire')
# change Chesterfield to East Derbyshire
chargepoint_number_df['ITL level 3'] = chargepoint_number_df['ITL level 3'].replace(['Chesterfield', 'North East Derbyshire', 'High Peak','Derbyshire'], 'East Derbyshire')
# change Chorley to Chorley and West Lancashire
chargepoint_number_df['ITL level 3'] = chargepoint_number_df['ITL level 3'].replace(['Chorley', 'Fylde', 'South Ribble', 'West Lancashire','Lancashire'], 'Chorley and West Lancashire')
# change City of London to Camden and City of London
chargepoint_number_df['ITL level 3'] = chargepoint_number_df['ITL level 3'].replace(['City of London', 'Camden', 'Lond','London','London Borough Of Southwark',
                                                               'London Borough of Brent', 'London Borough of Camden','London Borough of Ealing',
                                                               'London Borough of Enfield','London Borough of Greenwich', 'Royal Borough of Greenwich',
                                                               'London Borough of Hackney','London Borough of Hammersmith and Fulham',
                                                               'London Borough of Hounslow', 'London Borough of Islington',
                                                               'London Borough of Lambeth','London Borough of Lewisham',
                                                               'London Borough of Newham','London Borough of Richmond upon Thames',
                                                               'London Borough of Southwark','London Borough of Sutton',
                                                               'London Borough of Tower Hamlets','London Borough of Waltham Forest',
                                                               'London Borough of Wandsworth','Greater London'], 'Camden and City of London')
# change Clackmannanshire to Clackmannanshire and Fife
chargepoint_number_df['ITL level 3'] = chargepoint_number_df['ITL level 3'].replace(['Clackmannanshire', 'Fife'], 'Clackmannanshire and Fife')
# change Conwy to Conwy and Denbighshire
chargepoint_number_df['ITL level 3'] = chargepoint_number_df['ITL level 3'].replace(['Conwy', 'Denbighshire'], 'Conwy and Denbighshire')
# change Cornwall to Cornwall and Isles of Scilly
chargepoint_number_df['ITL level 3'] = chargepoint_number_df['ITL level 3'].replace(['Cornwall', 'Isles of Scilly'], 'Cornwall and Isles of Scilly')
# change County Durham to Durham CC
chargepoint_number_df.loc[chargepoint_number_df['ITL level 3'].str.contains('County Durham', case=False, na=False), 'ITL level 3'] = 'Durham CC'
# change Durham to Durham CC
chargepoint_number_df['ITL level 3'] = chargepoint_number_df['ITL level 3'].replace(['Durham','Co Durham'], 'Durham CC')
# change Crawley to West Sussex
chargepoint_number_df.loc[chargepoint_number_df['ITL level 3'].str.contains('Crawley', case=False, na=False), 'ITL level 3'] = 'West Sussex (North East)'
# change East Ayrshire to East Ayrshire and North Ayrshire mainland
chargepoint_number_df['ITL level 3'] = chargepoint_number_df['ITL level 3'].replace(['East Ayrshire', 'North Ayrshire'], 'East Ayrshire and North Ayrshire mainland')
# change East Devon to Devon CC
chargepoint_number_df['ITL level 3'] = chargepoint_number_df['ITL level 3'].replace(['East Devon', 'Exeter', 'Mid Devon', 'North Devon', 'South Hams', 'Teignbridge', 'Torridge', 'West Devon','Devon'], 'Devon CC')
# change East Dunbartonshire to East Dunbartonshire, West Dunbartonshire and Helensburgh and Lomond
chargepoint_number_df['ITL level 3'] = chargepoint_number_df['ITL level 3'].replace(['East Dunbartonshire', 'West dunbartonshire', 'West Dunbartonshire'], 'East Dunbartonshire, West Dunbartonshire and Helensburgh and Lomond')
# change East Hampshire to Central Hampshire
chargepoint_number_df['ITL level 3'] = chargepoint_number_df['ITL level 3'].replace(['East Hampshire', 'Eastleigh', 'New Forest'], 'Central Hampshire')
# change East Lothian to East Lothian and Midlothian
chargepoint_number_df['ITL level 3'] = chargepoint_number_df['ITL level 3'].replace(['East Lothian', 'Midlothian'], 'East Lothian and Midlothian')
# change East Renfrewshire to Inverclyde, East Renfrewshire and Renfrewshire
chargepoint_number_df['ITL level 3'] = chargepoint_number_df['ITL level 3'].replace(['East Renfrewshire', 'Renfewshire', 'Inverclyde', 'Renfrewshire'], 'Inverclyde, East Renfrewshire and Renfrewshire')
# change Eastbourne to East Sussex
chargepoint_number_df['ITL level 3'] = chargepoint_number_df['ITL level 3'].replace(['Eastbourne', 'East Sussex'])
# change Elmbridge to West Surrey
chargepoint_number_df['ITL level 3'] = chargepoint_number_df['ITL level 3'].replace(['Elmbridge', 'Epsom and Ewell', 'Guildford', 'Runnymede', 'Spelthorne', 'Woking', 'Waverley', 'Surrey Heath'], 'West Surrey')
# change Fareham to South Hampshire
chargepoint_number_df['ITL level 3'] = chargepoint_number_df['ITL level 3'].replace(['Fareham', 'Gosport', 'Havant', 'Hampshire'], 'South Hampshire')
# change Flintshire to Flintshire and Wrexham
chargepoint_number_df['ITL level 3'] = chargepoint_number_df['ITL level 3'].replace(['Flintshire', 'Wrexham'], 'Flintshire and Wrexham')
# change Folkestone and Hythe to East Kent
chargepoint_number_df['ITL level 3'] = chargepoint_number_df['ITL level 3'].replace(['Folkestone and Hythe', 'Dover', 'Ashford', 'Canterbury',
                                                                                     'KENT','Kent','kent'], 'East Kent')
# change Forest of Dean to Gloucestershire
chargepoint_number_df['ITL level 3'] = chargepoint_number_df['ITL level 3'].replace(['Gloucester', 'Forest of Dean', 'Cotswold', 'Cheltenham', 'Stroud', 'Tewkesbury'], 'Gloucestershire')
# change Gateshead to Tyneside
chargepoint_number_df['ITL level 3'] = chargepoint_number_df['ITL level 3'].replace(['Newcastle upon Tyne', 'Gateshead', 'North Tyneside', 'South Tyneside', 'Tyne & Wear', 'Tyne and Wear', 'Tyne and Wear,'], 'Tyneside')
# change Hackney to Hackney and Newham
chargepoint_number_df['ITL level 3'] = chargepoint_number_df['ITL level 3'].replace(['Hackney', 'Newham'], 'Hackney and Newham')
# change Hammersmith and Fulham to Kensington & Chelsea and Hammersmith & Fulham
chargepoint_number_df['ITL level 3'] = chargepoint_number_df['ITL level 3'].replace(['Hammersmith and Fulham', 'Hammersmith & Fulham Council', 'Kensington and Chelsea'], 'Kensington & Chelsea and Hammersmith & Fulham')
# change Harlow to Essex Thames Gateway
chargepoint_number_df['ITL level 3'] = chargepoint_number_df['ITL level 3'].replace(['Harlow', 'Epping Forest'], 'Essex Thames Gateway')
# change Harrow to Harrow and Hillingdon
chargepoint_number_df['ITL level 3'] = chargepoint_number_df['ITL level 3'].replace(['Harrow', 'Hillingdon'], 'Harrow and Hillingdon')
# change Haringey to Haringey and Islington
chargepoint_number_df['ITL level 3'] = chargepoint_number_df['ITL level 3'].replace(['Haringey', 'Islington'], 'Haringey and Islington')
# change Hastings to East Sussex CC
chargepoint_number_df['ITL level 3'] = chargepoint_number_df['ITL level 3'].replace(['Hastings', 'Lewes', 'Rother', 'Wealden','Sussex'], 'East Sussex CC')
# change Highland to Highlands and Islands
chargepoint_number_df.loc[chargepoint_number_df['ITL level 3'].str.contains('Highland', case=False, na=False), 'ITL level 3'] = 'Caithness and Sutherland and Ross and Cromarty'
# change Hounslow to Hounslow and Richmond upon Thames
chargepoint_number_df['ITL level 3'] = chargepoint_number_df['ITL level 3'].replace(['Hounslow', 'Richmond upon Thames'], 'Hounslow and Richmond upon Thames')
# change King's Lynn and West Norfolk to North and West Norfolk
chargepoint_number_df['ITL level 3'] = chargepoint_number_df['ITL level 3'].replace(["King's Lynn and West Norfolk", 'North Norfolk', 'norfolk'], 'North and West Norfolk')
# change Kingston upon Thames to Merton, Kingston upon Thames and Sutton
chargepoint_number_df['ITL level 3'] = chargepoint_number_df['ITL level 3'].replace(['Kingston upon Thames', 'Merton', 'Sutton'], 'Merton, Kingston upon Thames and Sutton')
# change Kirklees to Calderdale and Kirklees
chargepoint_number_df['ITL level 3'] = chargepoint_number_df['ITL level 3'].replace(['Kirklees', 'Calderdale'], 'Calderdale and Kirklees')
# change Knowsley to East Merseyside
chargepoint_number_df['ITL level 3'] = chargepoint_number_df['ITL level 3'].replace(['Knowsley', 'St. Helens'], 'East Merseyside')
# change Lancaster to Lancaster and Wyre
chargepoint_number_df['ITL level 3'] = chargepoint_number_df['ITL level 3'].replace(['Lancaster', 'Wyre', 'Wyre Forest'], 'Lancaster and Wyre')
# change Lewisham to Lewisham and Southwark
chargepoint_number_df['ITL level 3'] = chargepoint_number_df['ITL level 3'].replace(['Lewisham', 'Southwark'], 'Lewisham and Southwark')
# change Lincoln to North and North East Lincolnshire
chargepoint_number_df['ITL level 3'] = chargepoint_number_df['ITL level 3'].replace(['Lincoln', 'North East Lincolnshire', 'North Lincolnshire', 'East Lindsey'], 'North and North East Lincolnshire')
# change Maidstone to Medway
chargepoint_number_df['ITL level 3'] = chargepoint_number_df['ITL level 3'].replace('Maidstone', 'Medway')
# change Mansfield to Nottingham
chargepoint_number_df['ITL level 3'] = chargepoint_number_df['ITL level 3'].replace(['Mansfield', 'Newark and Sherwood','Nottinghamshire','City of Nottingham'], 'Nottingham')
# change Mendip to Somerset
chargepoint_number_df['ITL level 3'] = chargepoint_number_df['ITL level 3'].replace(['Mendip', 'South Somerset', 'Somerset West and Taunton', 'Sedgemoor'], 'Somerset')
# change Mole Valley to East Surrey
chargepoint_number_df['ITL level 3'] = chargepoint_number_df['ITL level 3'].replace(['Mole Valley', 'Reigate and Banstead', 'Tandridge'], 'East Surrey')
# change Monmouthshire to Monmouthshire and Newport
chargepoint_number_df['ITL level 3'] = chargepoint_number_df['ITL level 3'].replace(['Monmouthshire', 'Newport'], 'Monmouthshire and Newport')
# change Moray to Inverness and Nairn and Moray, Badenoch and Strathspey
chargepoint_number_df['ITL level 3'] = chargepoint_number_df['ITL level 3'].replace(['Moray'], 'Inverness and Nairn and Moray, Badenoch and Strathspey')
# change North Warwickshire to Warwickshire
chargepoint_number_df['ITL level 3'] = chargepoint_number_df['ITL level 3'].replace(['Warwick', 'North Warwickshire', 'Nuneaton and Bedworth', 'Rugby', 'Stratford-on-Avon'], 'Warwickshire')
# change Norwich to Norwich and East Norfolk
chargepoint_number_df.loc[chargepoint_number_df['ITL level 3'].str.contains('Norwich', case=False, na=False), 'ITL level 3'] = 'Norwich and East Norfolk'
# change Oldham to Greater Manchester North East
chargepoint_number_df['ITL level 3'] = chargepoint_number_df['ITL level 3'].replace(['Oldham', 'Rochdale', 'Salford'], 'Greater Manchester North East')
# change Perth and Kinross to Perth and Kinross and Stirling
chargepoint_number_df['ITL level 3'] = chargepoint_number_df['ITL level 3'].replace(['Perth and Kinross', 'Perth & Kinross', 'Perthshire', 'Stirling'], 'Perth and Kinross and Stirling')
# change preston to Mid Lancashire
chargepoint_number_df.loc[chargepoint_number_df['ITL level 3'].str.contains('Preston', case=False, na=False), 'ITL level 3'] = 'Mid Lancashire'
# change 'Redcar and Cleveland' to South Teesside
chargepoint_number_df['ITL level 3'] = chargepoint_number_df['ITL level 3'].replace(['Redcar and Cleveland','Cleveland'], 'South Teesside')
# change Redcar and Cleveland to North Yorkshire CC
chargepoint_number_df['ITL level 3'] = chargepoint_number_df['ITL level 3'].replace([ 'Yorkshire', 'North Yorkshire', 'South Yorkshire', 'West Yorkshire', 'west Yorkshire', 'Yorkshire, North Riding','Richmondshire', 'Middlesbrough', 'Hambleton', 'Harrogate', 'Ryedale', 'Scarborough','East Yorkshire', 'Selby', 'Craven'], 'North Yorkshire CC')
# change Redbridge to Redbridge and Waltham Forest
chargepoint_number_df['ITL level 3'] = chargepoint_number_df['ITL level 3'].replace(['Redbridge', 'Waltham Forest'], 'Redbridge and Waltham Forest')
# change Rushcliffe to Nottinghamshire
chargepoint_number_df.loc[chargepoint_number_df['ITL level 3'].str.contains('Rushcliffe', case=False, na=False), 'ITL level 3'] = 'South Nottinghamshire'
# change Shropshire to Shropshire CC
chargepoint_number_df.loc[chargepoint_number_df['ITL level 3'].str.contains('Shropshire', case=False, na=False), 'ITL level 3'] = 'Shropshire CC'
# change Stockport to Greater Manchester South East
chargepoint_number_df['ITL level 3'] = chargepoint_number_df['ITL level 3'].replace(['Stockport', 'Tameside', 'Wigan'], 'Greater Manchester South East')
# change 'Trafford' to Greater Manchester South West
chargepoint_number_df['ITL level 3'] = chargepoint_number_df['ITL level 3'].replace(['Trafford'], 'Greater Manchester South West')
# change Stockton-on-Tees to Hartlepool and Stockton-on-Tees
chargepoint_number_df['ITL level 3'] = chargepoint_number_df['ITL level 3'].replace(['Stockton-on-Tees', 'Hartlepool and Stockton-on-Tees'])
# change Tendring to Essex
chargepoint_number_df['ITL level 3'] = chargepoint_number_df['ITL level 3'].replace(['Essex','Tendring', 'Uttlesford'], 'West Essex')
# change Test Valley to Central Hampshire
chargepoint_number_df['ITL level 3'] = chargepoint_number_df['ITL level 3'].replace(['Isle Of Wight','Test Valley'], 'Central Hampshire')
# change Winchester to Portsmouth
chargepoint_number_df.loc[chargepoint_number_df['ITL level 3'].str.contains('Winchester', case=False, na=False), 'ITL level 3'] = 'Portsmouth'
# change  'Portsmouth City Council' to Portsmouth
chargepoint_number_df['ITL level 3'] = chargepoint_number_df['ITL level 3'].replace(['Portsmouth City Council'], 'Portsmouth')
# 'Tonbridge and Malling' to west Kent
chargepoint_number_df['ITL level 3'] = chargepoint_number_df['ITL level 3'].replace(['Folkestone','Tonbridge and Malling', 'Tunbridge Wells', 'Sevenoaks'], 'West Kent')
# change Tunbridge Wells to Kent Thames Gateway
chargepoint_number_df['ITL level 3'] = chargepoint_number_df['ITL level 3'].replace(['Thanet', 'Swale', 'Gravesham', 'Dartford'], 'Kent Thames Gateway')
# change Anglesey to Isle of Anglesey
chargepoint_number_df['ITL level 3'] = chargepoint_number_df['ITL level 3'].replace(['Anglesey'], 'Isle of Anglesey')
# change Antrim to Antrim and Newtownabbey
chargepoint_number_df['ITL level 3'] = chargepoint_number_df['ITL level 3'].replace(['Antrim', 'Newtownabbey', 'County Antrim'], 'Antrim and Newtownabbey')
# change Armagh to Armagh City, Banbridge and Craigavon
chargepoint_number_df['ITL level 3'] = chargepoint_number_df['ITL level 3'].replace(['Armagh', 'County Armagh'], 'Armagh City, Banbridge and Craigavon')
# change Attleborough to Breckland and South Norfolk
chargepoint_number_df['ITL level 3'] = chargepoint_number_df['ITL level 3'].replace(['Attleborough'], 'Breckland and South Norfolk')
# change Avon to City of, Bristol
chargepoint_number_df['ITL level 3'] = chargepoint_number_df['ITL level 3'].replace(['County of Bristol','Avon', 'Bristol', 'City of Bristol'], 'Bristol, City of')
# change Bedfordshire to Central Bedfordshire
chargepoint_number_df['ITL level 3'] = chargepoint_number_df['ITL level 3'].replace(['Bedfordshire', 'Beds','Dunstable'], 'Central Bedfordshire')
# change Belfast Greater to Belfast
chargepoint_number_df['ITL level 3'] = chargepoint_number_df['ITL level 3'].replace(['Belfast Greater'], 'Belfast')
# change Bilston to Wolverhampton
chargepoint_number_df['ITL level 3'] = chargepoint_number_df['ITL level 3'].replace(['Bilston'], 'Wolverhampton')
# change Bournemouth to Bournemouth, Christchurch and Poole
chargepoint_number_df['ITL level 3'] = chargepoint_number_df['ITL level 3'].replace(['Bournemouth'], 'Bournemouth, Christchurch and Poole')
# change Calne to Wiltshire
chargepoint_number_df['ITL level 3'] = chargepoint_number_df['ITL level 3'].replace(['Calne'], 'Wiltshire')
# change carmarhenshire to South West Wales
chargepoint_number_df['ITL level 3'] = chargepoint_number_df['ITL level 3'].replace(['Carmarhenshire','Dyfed'], 'South West Wales')
# change Cheshire to Cheshire East
chargepoint_number_df['ITL level 3'] = chargepoint_number_df['ITL level 3'].replace(['Cheshire','Deeside'], 'Cheshire East')
# change Coventry to City of Coventry
chargepoint_number_df['ITL level 3'] = chargepoint_number_df['ITL level 3'].replace(['City of Coventry'], 'Coventry')
# change City of Glasgow to Glasgow City
chargepoint_number_df['ITL level 3'] = chargepoint_number_df['ITL level 3'].replace(['City of Glasgow','Glasgow'], 'Glasgow City')
# change Dumfries & Galloway to Dumfries and Galloway
chargepoint_number_df['ITL level 3'] = chargepoint_number_df['ITL level 3'].replace(['Dumfries & Galloway'], 'Dumfries and Galloway')
# change Clwyd to Flintshire and Wrexham
chargepoint_number_df['ITL level 3'] = chargepoint_number_df['ITL level 3'].replace(['Clwyd'], 'Flintshire and Wrexham')
# change County Down to Newry, Mourne and Down
chargepoint_number_df['ITL level 3'] = chargepoint_number_df['ITL level 3'].replace(['County Down','Down'], 'Newry, Mourne and Down')  
# change County Fermanagh to Fermanagh and Omagh
chargepoint_number_df['ITL level 3'] = chargepoint_number_df['ITL level 3'].replace(['County Fermanagh','Fermanagh'], 'Fermanagh and Omagh')
# change County Londonderry to Derry City and Strabane
chargepoint_number_df['ITL level 3'] = chargepoint_number_df['ITL level 3'].replace(['County Londonderry','Londonderry'], 'Derry City and Strabane')
# change County Tyrone to Mid Ulster
chargepoint_number_df['ITL level 3'] = chargepoint_number_df['ITL level 3'].replace(['County Tyrone','Tyrone'], 'Mid Ulster')
# change Derry to Derry City and Strabane
chargepoint_number_df['ITL level 3'] = chargepoint_number_df['ITL level 3'].replace(['Derry'], 'Derry City and Strabane')
# change Dumfriesshire to Dumfries and Galloway
chargepoint_number_df['ITL level 3'] = chargepoint_number_df['ITL level 3'].replace(['Dumfriesshire'], 'Dumfries and Galloway')
# change Dunbartonshire to East Dunbartonshire, West Dunbartonshire and Helensburgh and Lomond
chargepoint_number_df['ITL level 3'] = chargepoint_number_df['ITL level 3'].replace(['Dunbartonshire'], 'East Dunbartonshire, West Dunbartonshire and Helensburgh and Lomond')
# change Edinburgh to City of Edinburgh
chargepoint_number_df['ITL level 3'] = chargepoint_number_df['ITL level 3'].replace(['Edinburgh'], 'City of Edinburgh')
# change Eilean Siar to Na h-Eileanan Siar
chargepoint_number_df['ITL level 3'] = chargepoint_number_df['ITL level 3'].replace(['Eilean Siar'], 'Na h-Eileanan Siar')
# change  Hull City to Kingston upon Hull, City of
chargepoint_number_df['ITL level 3'] = chargepoint_number_df['ITL level 3'].replace(['Hull City'], 'Kingston upon Hull, City of')



# check which values are in ITL level 3 in chargepoint_number_df but not in gdhi_population_fuel_electric_df
remove_rows = set(chargepoint_number_df['ITL level 3'].unique()) - set(gdhi_population_fuel_electric_df['ITL level 3'].unique())

# remove rows with values in ITL level 3 in chargepoint_number_df but not in gdhi_population_fuel_electric_df
chargepoint_number_df = chargepoint_number_df[~chargepoint_number_df['ITL level 3'].isin(remove_rows)]

# group by and aggregate by sum and year
chargepoint_number_df = chargepoint_number_df.groupby(['ITL level 3', 'Year','Quarter']).sum().reset_index()



In [151]:
chargepoint_number_df

,ITL level 3,Year,Quarter,Number of Chargepoints
0,Aberdeen City and Aberdeenshire,2015,1,15
1,Aberdeen City and Aberdeenshire,2015,2,7
2,Aberdeen City and Aberdeenshire,2015,3,11
3,Aberdeen City and Aberdeenshire,2015,4,3
4,Aberdeen City and Aberdeenshire,2016,1,5
...,...,...,...,...
1226,Worcestershire,2023,1,4
1227,Worcestershire,2023,2,6
1228,York,2018,1,2
1229,York,2018,3,1


In [152]:
# Create a new empty column called Sum of Chargepoints
chargepoint_number_df['Sum of Chargepoints'] = 0

# Sort by 'ITL level 3' and 'Year'
chargepoint_number_df.sort_values(by=['ITL level 3', 'Year', 'Quarter'], inplace=True)

# Reset the index of dataframe to handle index issues while using .shift()
chargepoint_number_df.reset_index(drop=True, inplace=True)

# Group by ITL level 3
grouped = chargepoint_number_df.groupby('ITL level 3')

# Iterate over groups
for name, group in grouped:
    # Create a new column in the group dataframe for the cumulative sum of 'Number of Chargepoints'
    group['Sum of Chargepoints'] = group['Number of Chargepoints'].cumsum()

    # Update the original dataframe
    chargepoint_number_df.update(group)

chargepoint_number_df.head(20)


,ITL level 3,Year,Quarter,Number of Chargepoints,Sum of Chargepoints
0,Aberdeen City and Aberdeenshire,2015.0,1.0,15.0,15.0
1,Aberdeen City and Aberdeenshire,2015.0,2.0,7.0,22.0
2,Aberdeen City and Aberdeenshire,2015.0,3.0,11.0,33.0
3,Aberdeen City and Aberdeenshire,2015.0,4.0,3.0,36.0
4,Aberdeen City and Aberdeenshire,2016.0,1.0,5.0,41.0
5,Aberdeen City and Aberdeenshire,2016.0,4.0,7.0,48.0
6,Aberdeen City and Aberdeenshire,2017.0,4.0,2.0,50.0
7,Aberdeen City and Aberdeenshire,2018.0,3.0,1.0,51.0
8,Aberdeen City and Aberdeenshire,2019.0,1.0,2.0,53.0
9,Aberdeen City and Aberdeenshire,2019.0,2.0,2.0,55.0


In [153]:
# define the list of dataframes
#frames = [battery_electric_gdhi_df, plug_in_hybrid_diesel_gdhi_df, plug_in_hybrid_petrol_gdhi_df, range_extended_electric_gdhi_df, hybrid_petrol_gdhi_df]

# define the batch size for concatenation
#batch_size = 10000

# initialize an empty list to store the concatenated dataframes
#concatenated_df = []

# iterate over the frames in batches and concatenate
#for i in range(0, len(frames), batch_size):
    #batch_frames = frames[i:i+batch_size]
    #concatenated_batch = pd.concat(batch_frames)
    #concatenated_df.append(concatenated_batch)

# concatenate the batches into a final dataframe
#all_vehicles_electric__df = pd.concat(concatenated_df)

# replace with total
all_vehicles_electric__df = total_gdhi_df

# if Units column contains values other than 'Number', multiply the value of "Number of vehicles" by 1000 using dictionary
all_vehicles_electric__df['Number of vehicles'] = np.where(all_vehicles_electric__df['Units'] != 'Number', all_vehicles_electric__df['Number of vehicles'] * 1000, all_vehicles_electric__df['Number of vehicles'])

# drop Units column
all_vehicles_electric__df.drop(['Units'], axis=1, inplace=True)

In [154]:
# create a new column called 'Sum of Chargepoints' and set it to 0
all_vehicles_electric__df['Sum of Chargepoints'] = 0

In [155]:
# Create a dictionary with ITL level 3 and Year as key and sum of chargepoints as value
chargepoint_dict = chargepoint_number_df.set_index(['ITL level 3', 'Year','Quarter'])['Sum of Chargepoints'].to_dict()

# Apply the dictionary to the all_vehicles_electric__df dataframe
all_vehicles_electric__df['Sum of Chargepoints'] = all_vehicles_electric__df.set_index(['ITL level 3', 'Year','Quarter']).index.map(chargepoint_dict).fillna(0).astype(int)

In [156]:
# drop PES area
all_vehicles_electric__df.drop(['PES area','ITL level'], axis=1, inplace=True)
# drop column ITL code
all_vehicles_electric__df.drop(['ITL code'], axis=1, inplace=True)
all_vehicles_electric__df.columns

Index(['ITL level 3', 'Year', 'gdhi', 'sex', 'age', 'population',
       'ITL level 1', 'ITL level 2', 'Average variable unit price (£/kWh)',
       ' ULSP:  Pump price (p/litre)', 'ULSD: Pump price (p/litre)',
       'BodyType', 'Fuel', 'Keepership', 'Number of vehicles', 'Quarter',
       'Sum of Chargepoints'],
      dtype='object')

In [157]:
# get education data from csv file 
education_df_2010 = pd.read_excel('data/Education/vol02_2011c3.xls', sheet_name='3.8', skiprows=11)
# drop first 3 columns
education_df_2010.drop(education_df_2010.columns[:3], axis=1, inplace=True)
# drop only rows with NaN values
education_df_2010.dropna(axis=0, how='all', inplace=True)
# drop columns with NaN values
education_df_2010.dropna(axis=1, how='all', inplace=True)
# drop rows with NaN values in column 1
education_df_2010.dropna(subset=education_df_2010.columns[0], inplace=True)
# rename first column to 'ITL level 1'
education_df_2010.rename(columns={education_df_2010.columns[0]: 'ITL level 1'}, inplace=True)
# drop 2nd and 5th columns
education_df_2010.drop(education_df_2010.columns[[1,4]], axis=1, inplace=True)


# rename columns
education_df_2010.rename(columns={'NQF level 4 or above2': 'NQF level 4 or above', 
                             'NQF level 3 or above3': 'NQF level 3 or above', 'NQF level 2 or above4': 'NQF level 2 or above'}, 
                             inplace=True)
regions = ['London', 'Yorkshire and The Humber', 'South West', 'East of England', 'South East', 'West Midlands', 'North West', 'North East', 'East Midlands']
# create new rows for regions
education_df_2010 = education_df_2010.append(pd.DataFrame({'ITL level 1': regions}), ignore_index=True)
# set values for NQF level 4 or above for regions with England value
education_df_2010.loc[education_df_2010['ITL level 1'].isin(regions), 'NQF level 4 or above'] = education_df_2010.loc[education_df_2010['ITL level 1'] == 'England', 'NQF level 4 or above'].values[0]
# set values for NQF level 3 or above for regions with England value
education_df_2010.loc[education_df_2010['ITL level 1'].isin(regions), 'NQF level 3 or above'] = education_df_2010.loc[education_df_2010['ITL level 1'] == 'England', 'NQF level 3 or above'].values[0]
# set values for NQF level 2 or above for regions with England value
education_df_2010.loc[education_df_2010['ITL level 1'].isin(regions), 'NQF level 2 or above'] = education_df_2010.loc[education_df_2010['ITL level 1'] == 'England', 'NQF level 2 or above'].values[0]
# add year column
education_df_2010['Year'] = 2010
education_df_2010


/var/folders/j9/0qd6cw014pgb7rk0v_28prrh0000gn/T/ipykernel_15876/3805673639.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  education_df_2010 = education_df_2010.append(pd.DataFrame({'ITL level 1': regions}), ignore_index=True)


,ITL level 1,NQF level 4 or above,NQF level 3 or above,NQF level 2 or above,Year
0,England,38.0,59.0,78.0,2010
1,Wales,32.0,53.0,74.0,2010
2,Scotland,37.0,58.0,77.0,2010
3,Northern Ireland,31.0,51.0,71.0,2010
4,London,38.0,59.0,78.0,2010
5,Yorkshire and The Humber,38.0,59.0,78.0,2010
6,South West,38.0,59.0,78.0,2010
7,East of England,38.0,59.0,78.0,2010
8,South East,38.0,59.0,78.0,2010
9,West Midlands,38.0,59.0,78.0,2010


In [158]:
# get education data from csv file
education_df_2012 = pd.read_excel('data/Education/v01-2013c3.xls', sheet_name='3.4', skiprows=11)
# drop first 3 columns
education_df_2012.drop(education_df_2012.columns[:3], axis=1, inplace=True)
# drop only rows with NaN values
education_df_2012.dropna(axis=0, how='all', inplace=True)
# drop columns with NaN values
education_df_2012.dropna(axis=1, how='all', inplace=True)
# drop rows with NaN values in column 1
education_df_2012.dropna(subset=education_df_2012.columns[0], inplace=True)
# rename first column to 'ITL level 1'
education_df_2012.rename(columns={education_df_2012.columns[0]: 'ITL level 1'}, inplace=True)
# drop 2nd,5th and 7th columns
education_df_2012.drop(education_df_2012.columns[[1,4,6]], axis=1, inplace=True)
# rename columns
education_df_2012.rename(columns={'Unnamed: 9': 'NQF level 4 or above',
                                'Unnamed: 11': 'NQF level 3 or above', 
                                'Unnamed: 14': 'NQF level 2 or above'},
                                inplace=True)
# rename rows with values 'England6' to 'England'
education_df_2012.loc[education_df_2012['ITL level 1'] == 'England6', 'ITL level 1'] = 'England'
# create new rows for regions
education_df_2012 = education_df_2012.append(pd.DataFrame({'ITL level 1': regions}), ignore_index=True)
# set values for NQF level 4 or above for regions with England value
education_df_2012.loc[education_df_2012['ITL level 1'].isin(regions), 'NQF level 4 or above'] = education_df_2012.loc[education_df_2012['ITL level 1'] == 'England', 'NQF level 4 or above'].values[0]
# set values for NQF level 3 or above for regions with England value
education_df_2012.loc[education_df_2012['ITL level 1'].isin(regions), 'NQF level 3 or above'] = education_df_2012.loc[education_df_2012['ITL level 1'] == 'England', 'NQF level 3 or above'].values[0]
# set values for NQF level 2 or above for regions with England value
education_df_2012.loc[education_df_2012['ITL level 1'].isin(regions), 'NQF level 2 or above'] = education_df_2012.loc[education_df_2012['ITL level 1'] == 'England', 'NQF level 2 or above'].values[0]
# add year column
education_df_2012['Year'] = 2012

education_df_2012

/var/folders/j9/0qd6cw014pgb7rk0v_28prrh0000gn/T/ipykernel_15876/3181929227.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  education_df_2012 = education_df_2012.append(pd.DataFrame({'ITL level 1': regions}), ignore_index=True)


,ITL level 1,NQF level 4 or above,NQF level 3 or above,NQF level 2 or above,Year
0,England,39,61,81,2012
1,Wales,33,54,75,2012
2,Northern Ireland,31,50,70,2012
3,Scotland,42,65,82,2012
4,London,39,61,81,2012
5,Yorkshire and The Humber,39,61,81,2012
6,South West,39,61,81,2012
7,East of England,39,61,81,2012
8,South East,39,61,81,2012
9,West Midlands,39,61,81,2012


In [159]:
# get education data from csv file 
education_df_2013 = pd.read_excel('data/Education/Tables_SR45_2014.xls', sheet_name='3.4', skiprows=11)
# drop first 3 columns
education_df_2013.drop(education_df_2013.columns[:3], axis=1, inplace=True)
# drop only rows with NaN values
education_df_2013.dropna(axis=0, how='all', inplace=True)
# drop columns with NaN values
education_df_2013.dropna(axis=1, how='all', inplace=True)
# drop rows with NaN values in column 1
education_df_2013.dropna(subset=education_df_2013.columns[0], inplace=True)
# rename first column to 'ITL level 1'
education_df_2013.rename(columns={education_df_2013.columns[0]: 'ITL level 1'}, inplace=True)
# drop 2nd and 5th columns
education_df_2013.drop(education_df_2013.columns[[1,4, 6]], axis=1, inplace=True)
# rename columns
education_df_2013.rename(columns={'Unnamed: 9': 'NQF level 4 or above',
                                'Unnamed: 11': 'NQF level 3 or above', 
                                'Unnamed: 14': 'NQF level 2 or above'},
                                inplace=True)

# rename rows with values 'England6' to 'England'
education_df_2013.loc[education_df_2013['ITL level 1'] == 'England6', 'ITL level 1'] = 'England'
# create new rows for regions
education_df_2013 = education_df_2013.append(pd.DataFrame({'ITL level 1': regions}), ignore_index=True)
# set values for NQF level 4 or above for regions with England value
education_df_2013.loc[education_df_2013['ITL level 1'].isin(regions), 'NQF level 4 or above'] = education_df_2013.loc[education_df_2013['ITL level 1'] == 'England', 'NQF level 4 or above'].values[0]
# set values for NQF level 3 or above for regions with England value
education_df_2013.loc[education_df_2013['ITL level 1'].isin(regions), 'NQF level 3 or above'] = education_df_2013.loc[education_df_2013['ITL level 1'] == 'England', 'NQF level 3 or above'].values[0]
# set values for NQF level 2 or above for regions with England value
education_df_2013.loc[education_df_2013['ITL level 1'].isin(regions), 'NQF level 2 or above'] = education_df_2013.loc[education_df_2013['ITL level 1'] == 'England', 'NQF level 2 or above'].values[0]
# add year column
education_df_2013['Year'] = 2013

education_df_2013


/var/folders/j9/0qd6cw014pgb7rk0v_28prrh0000gn/T/ipykernel_15876/2262008558.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  education_df_2013 = education_df_2013.append(pd.DataFrame({'ITL level 1': regions}), ignore_index=True)


,ITL level 1,NQF level 4 or above,NQF level 3 or above,NQF level 2 or above,Year
0,England,40.26886,62.519566,80.58216,2013
1,Wales,33.904537,55.51364,76.717175,2013
2,Northern Ireland,31.061537,50.193683,71.279107,2013
3,Scotland,43,67,83,2013
4,London,40.26886,62.519566,80.58216,2013
5,Yorkshire and The Humber,40.26886,62.519566,80.58216,2013
6,South West,40.26886,62.519566,80.58216,2013
7,East of England,40.26886,62.519566,80.58216,2013
8,South East,40.26886,62.519566,80.58216,2013
9,West Midlands,40.26886,62.519566,80.58216,2013


In [160]:
# get education data from csv file
education_df_2014 = pd.read_excel('data/Education/SR43_2015_Tables.xlsx', sheet_name='3.4', skiprows=11)
# drop first 3 columns
education_df_2014.drop(education_df_2014.columns[:3], axis=1, inplace=True)
# drop only rows with NaN values
education_df_2014.dropna(axis=0, how='all', inplace=True)
# drop columns with NaN values
education_df_2014.dropna(axis=1, how='all', inplace=True)
# drop rows with NaN values in column 1
education_df_2014.dropna(subset=education_df_2014.columns[0], inplace=True)
# rename first column to 'ITL level 1'
education_df_2014.rename(columns={education_df_2014.columns[0]: 'ITL level 1'}, inplace=True)
# drop 2nd columns
education_df_2014.drop(education_df_2014.columns[[1]], axis=1, inplace=True)
# rename columns
education_df_2014.rename(columns={'Unnamed: 8': 'NQF level 4 or above',
                                'Unnamed: 10': 'NQF level 3 or above',
                                'Unnamed: 12': 'NQF level 2 or above'},
                                inplace=True)

# rename rows with values 'England6' to 'England'
education_df_2014.loc[education_df_2014['ITL level 1'] == 'England6', 'ITL level 1'] = 'England'
# create new rows for regions
education_df_2014 = education_df_2014.append(pd.DataFrame({'ITL level 1': regions}), ignore_index=True)
# set values for NQF level 4 or above for regions with England value

education_df_2014.loc[education_df_2014['ITL level 1'].isin(regions), 'NQF level 4 or above'] = education_df_2014.loc[education_df_2014['ITL level 1'] == 'England', 'NQF level 4 or above'].values[0]
# set values for NQF level 3 or above for regions with England value
education_df_2014.loc[education_df_2014['ITL level 1'].isin(regions), 'NQF level 3 or above'] = education_df_2014.loc[education_df_2014['ITL level 1'] == 'England', 'NQF level 3 or above'].values[0]
# set values for NQF level 2 or above for regions with England value
education_df_2014.loc[education_df_2014['ITL level 1'].isin(regions), 'NQF level 2 or above'] = education_df_2014.loc[education_df_2014['ITL level 1'] == 'England', 'NQF level 2 or above'].values[0]
# add year column
education_df_2014['Year'] = 2014

education_df_2014

/var/folders/j9/0qd6cw014pgb7rk0v_28prrh0000gn/T/ipykernel_15876/531055259.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  education_df_2014 = education_df_2014.append(pd.DataFrame({'ITL level 1': regions}), ignore_index=True)


,ITL level 1,NQF level 4 or above,NQF level 3 or above,NQF level 2 or above,Year
0,England,41,63,81,2014
1,Wales,36,56,76,2014
2,Northern Ireland,32,51,72,2014
3,Scotland,45,67,83,2014
4,London,41,63,81,2014
5,Yorkshire and The Humber,41,63,81,2014
6,South West,41,63,81,2014
7,East of England,41,63,81,2014
8,South East,41,63,81,2014
9,West Midlands,41,63,81,2014


In [161]:
# get education data from csv file
education_df_2015 = pd.read_excel('data/Education/SR54_2016_Tables.xlsx', sheet_name='3.4', skiprows=11)
# drop first 3 columns
education_df_2015.drop(education_df_2015.columns[:3], axis=1, inplace=True)
# drop only rows with NaN values
education_df_2015.dropna(axis=0, how='all', inplace=True)
# drop columns with NaN values
education_df_2015.dropna(axis=1, how='all', inplace=True)
# drop rows with NaN values in column 1
education_df_2015.dropna(subset=education_df_2015.columns[0], inplace=True)
# rename first column to 'ITL level 1'
education_df_2015.rename(columns={education_df_2015.columns[0]: 'ITL level 1'}, inplace=True)
# drop 2nd columns
education_df_2015.drop(education_df_2015.columns[[1]], axis=1, inplace=True)
# rename columns
education_df_2015.rename(columns={'Unnamed: 8': 'NQF level 4 or above',
                                'Unnamed: 10': 'NQF level 3 or above',
                                'Unnamed: 12': 'NQF level 2 or above'},
                                inplace=True)

# rename rows with values 'England6' to 'England'
education_df_2015.loc[education_df_2015['ITL level 1'] == 'England6', 'ITL level 1'] = 'England'
# rename rows with values 'Scotland (2015)' to 'Scotland'
education_df_2015.loc[education_df_2015['ITL level 1'] == 'Scotland (2015)', 'ITL level 1'] = 'Scotland'
# create new rows for regions
education_df_2015 = education_df_2015.append(pd.DataFrame({'ITL level 1': regions}), ignore_index=True)
# set values for NQF level 4 or above for regions with England value
education_df_2015.loc[education_df_2015['ITL level 1'].isin(regions), 'NQF level 4 or above'] = education_df_2015.loc[education_df_2015['ITL level 1'] == 'England', 'NQF level 4 or above'].values[0]
# set values for NQF level 3 or above for regions with England value
education_df_2015.loc[education_df_2015['ITL level 1'].isin(regions), 'NQF level 3 or above'] = education_df_2015.loc[education_df_2015['ITL level 1'] == 'England', 'NQF level 3 or above'].values[0]
# set values for NQF level 2 or above for regions with England value
education_df_2015.loc[education_df_2015['ITL level 1'].isin(regions), 'NQF level 2 or above'] = education_df_2015.loc[education_df_2015['ITL level 1'] == 'England', 'NQF level 2 or above'].values[0]
# add year column
education_df_2015['Year'] = 2015

education_df_2015

/var/folders/j9/0qd6cw014pgb7rk0v_28prrh0000gn/T/ipykernel_15876/4230302642.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  education_df_2015 = education_df_2015.append(pd.DataFrame({'ITL level 1': regions}), ignore_index=True)


,ITL level 1,NQF level 4 or above,NQF level 3 or above,NQF level 2 or above,Year
0,England,41,63,81,2015
1,Wales,36,56,76,2015
2,Northern Ireland,32,51,72,2015
3,Scotland,46,70,77,2015
4,London,41,63,81,2015
5,Yorkshire and The Humber,41,63,81,2015
6,South West,41,63,81,2015
7,East of England,41,63,81,2015
8,South East,41,63,81,2015
9,West Midlands,41,63,81,2015


In [162]:
# get education data from csv file
education_df_2016 = pd.read_excel('data/Education/SFR64_2017_Tables.xlsx', sheet_name='3.4', skiprows=11)
# drop first 3 columns
education_df_2016.drop(education_df_2016.columns[:3], axis=1, inplace=True)
# drop only rows with NaN values
education_df_2016.dropna(axis=0, how='all', inplace=True)
# drop columns with NaN values
education_df_2016.dropna(axis=1, how='all', inplace=True)
# drop rows with NaN values in column 1
education_df_2016.dropna(subset=education_df_2016.columns[0], inplace=True)
# rename first column to 'ITL level 1'
education_df_2016.rename(columns={education_df_2016.columns[0]: 'ITL level 1'}, inplace=True)
# drop 2nd columns
education_df_2016.drop(education_df_2016.columns[[1]], axis=1, inplace=True)
# rename columns
education_df_2016.rename(columns={'Unnamed: 8': 'NQF level 4 or above',
                                'Unnamed: 10': 'NQF level 3 or above',
                                'Unnamed: 12': 'NQF level 2 or above'},
                                inplace=True)

# rename rows with values 'England6' to 'England'
education_df_2016.loc[education_df_2016['ITL level 1'] == 'England6', 'ITL level 1'] = 'England'
# create new rows for regions
education_df_2016 = education_df_2016.append(pd.DataFrame({'ITL level 1': regions}), ignore_index=True)
# set values for NQF level 4 or above for regions with England value
education_df_2016.loc[education_df_2016['ITL level 1'].isin(regions), 'NQF level 4 or above'] = education_df_2016.loc[education_df_2016['ITL level 1'] == 'England', 'NQF level 4 or above'].values[0]
# set values for NQF level 3 or above for regions with England value
education_df_2016.loc[education_df_2016['ITL level 1'].isin(regions), 'NQF level 3 or above'] = education_df_2016.loc[education_df_2016['ITL level 1'] == 'England', 'NQF level 3 or above'].values[0]
# set values for NQF level 2 or above for regions with England value
education_df_2016.loc[education_df_2016['ITL level 1'].isin(regions), 'NQF level 2 or above'] = education_df_2016.loc[education_df_2016['ITL level 1'] == 'England', 'NQF level 2 or above'].values[0]
# add year column
education_df_2016['Year'] = 2016

education_df_2016

/var/folders/j9/0qd6cw014pgb7rk0v_28prrh0000gn/T/ipykernel_15876/921792203.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  education_df_2016 = education_df_2016.append(pd.DataFrame({'ITL level 1': regions}), ignore_index=True)


,ITL level 1,NQF level 4 or above,NQF level 3 or above,NQF level 2 or above,Year
0,England,43.9,65.2,82.9,2016
1,Wales,37.2,57.6,78.6,2016
2,Northern Ireland,35.2,53.9,73.2,2016
3,Scotland,47,69,78,2016
4,London,43.9,65.2,82.9,2016
5,Yorkshire and The Humber,43.9,65.2,82.9,2016
6,South West,43.9,65.2,82.9,2016
7,East of England,43.9,65.2,82.9,2016
8,South East,43.9,65.2,82.9,2016
9,West Midlands,43.9,65.2,82.9,2016


In [163]:
# get education data from csv file
education_df_2017 = pd.read_excel('data/Education/UKETS_2018_Tables.xlsx', sheet_name='3.4', skiprows=11)
# drop first 3 columns
education_df_2017.drop(education_df_2017.columns[:3], axis=1, inplace=True)
# drop only rows with NaN values
education_df_2017.dropna(axis=0, how='all', inplace=True)
# drop columns with NaN values
education_df_2017.dropna(axis=1, how='all', inplace=True)
# drop rows with NaN values in column 1
education_df_2017.dropna(subset=education_df_2017.columns[0], inplace=True)
# rename first column to 'ITL level 1'
education_df_2017.rename(columns={education_df_2017.columns[0]: 'ITL level 1'}, inplace=True)
# drop 2nd columns
education_df_2017.drop(education_df_2017.columns[[1]], axis=1, inplace=True)
# rename columns
education_df_2017.rename(columns={'Unnamed: 8': 'NQF level 4 or above',
                                'Unnamed: 10': 'NQF level 3 or above',
                                'Unnamed: 12': 'NQF level 2 or above'},
                                inplace=True)

# rename rows with values 'England6' to 'England'
education_df_2017.loc[education_df_2017['ITL level 1'] == 'England6', 'ITL level 1'] = 'England'
# create new rows for regions
education_df_2017 = education_df_2017.append(pd.DataFrame({'ITL level 1': regions}), ignore_index=True)
# set values for NQF level 4 or above for regions with England value
education_df_2017.loc[education_df_2017['ITL level 1'].isin(regions), 'NQF level 4 or above'] = education_df_2017.loc[education_df_2017['ITL level 1'] == 'England', 'NQF level 4 or above'].values[0]
# set values for NQF level 3 or above for regions with England value
education_df_2017.loc[education_df_2017['ITL level 1'].isin(regions), 'NQF level 3 or above'] = education_df_2017.loc[education_df_2017['ITL level 1'] == 'England', 'NQF level 3 or above'].values[0]
# set values for NQF level 2 or above for regions with England value
education_df_2017.loc[education_df_2017['ITL level 1'].isin(regions), 'NQF level 2 or above'] = education_df_2017.loc[education_df_2017['ITL level 1'] == 'England', 'NQF level 2 or above'].values[0]
# add year column
education_df_2017['Year'] = 2017

education_df_2017

/var/folders/j9/0qd6cw014pgb7rk0v_28prrh0000gn/T/ipykernel_15876/2125527377.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  education_df_2017 = education_df_2017.append(pd.DataFrame({'ITL level 1': regions}), ignore_index=True)


,ITL level 1,NQF level 4 or above,NQF level 3 or above,NQF level 2 or above,Year
0,England,43.9,66,82.9,2017
1,Wales,37.2,59,78,2017
2,Northern Ireland,34,53.9,74,2017
3,Scotland,47,70,78,2017
4,London,43.9,66,82.9,2017
5,Yorkshire and The Humber,43.9,66,82.9,2017
6,South West,43.9,66,82.9,2017
7,East of England,43.9,66,82.9,2017
8,South East,43.9,66,82.9,2017
9,West Midlands,43.9,66,82.9,2017


In [164]:
# get education data from csv file
education_df_2018 = pd.read_excel('data/Education/UKETS_2019_Tables.xlsx', sheet_name='3.3', skiprows=11)
# drop first 3 columns
education_df_2018.drop(education_df_2018.columns[:3], axis=1, inplace=True)
# drop only rows with NaN values
education_df_2018.dropna(axis=0, how='all', inplace=True)
# drop columns with NaN values
education_df_2018.dropna(axis=1, how='all', inplace=True)
# drop rows with NaN values in column 1
education_df_2018.dropna(subset=education_df_2018.columns[0], inplace=True)
# rename first column to 'ITL level 1'
education_df_2018.rename(columns={education_df_2018.columns[0]: 'ITL level 1'}, inplace=True)
# drop 2nd columns
education_df_2018.drop(education_df_2018.columns[[1]], axis=1, inplace=True)
# rename columns
education_df_2018.rename(columns={'Unnamed: 8': 'NQF level 4 or above',
                                'Unnamed: 10': 'NQF level 3 or above',
                                'Unnamed: 12': 'NQF level 2 or above'},
                                inplace=True)

# rename rows with values 'England6' to 'England'
education_df_2018.loc[education_df_2018['ITL level 1'] == 'England(5)', 'ITL level 1'] = 'England'
# create new rows for regions
education_df_2018 = education_df_2018.append(pd.DataFrame({'ITL level 1': regions}), ignore_index=True)
# set values for NQF level 4 or above for regions with England value
education_df_2018.loc[education_df_2018['ITL level 1'].isin(regions), 'NQF level 4 or above'] = education_df_2018.loc[education_df_2018['ITL level 1'] == 'England', 'NQF level 4 or above'].values[0]
# set values for NQF level 3 or above for regions with England value
education_df_2018.loc[education_df_2018['ITL level 1'].isin(regions), 'NQF level 3 or above'] = education_df_2018.loc[education_df_2018['ITL level 1'] == 'England', 'NQF level 3 or above'].values[0]
# set values for NQF level 2 or above for regions with England value
education_df_2018.loc[education_df_2018['ITL level 1'].isin(regions), 'NQF level 2 or above'] = education_df_2018.loc[education_df_2018['ITL level 1'] == 'England', 'NQF level 2 or above'].values[0]
# add year column
education_df_2018['Year'] = 2018

education_df_2018


/var/folders/j9/0qd6cw014pgb7rk0v_28prrh0000gn/T/ipykernel_15876/1943381949.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  education_df_2018 = education_df_2018.append(pd.DataFrame({'ITL level 1': regions}), ignore_index=True)


,ITL level 1,NQF level 4 or above,NQF level 3 or above,NQF level 2 or above,Year
0,England,42,62,81,2018
1,Wales,39,60,81,2018
2,Northern Ireland,37,57,77,2018
3,Scotland,49,68,79,2018
4,London,42,62,81,2018
5,Yorkshire and The Humber,42,62,81,2018
6,South West,42,62,81,2018
7,East of England,42,62,81,2018
8,South East,42,62,81,2018
9,West Midlands,42,62,81,2018


In [165]:
# read the csv file
education_df_2019 = pd.read_csv('data/Education/year_2019.csv')
# rename columns
education_df_2019.rename(columns={'pt_hiqual_nqf4': 'NQF level 4 or above',
                                'pt_hiqual_nqf3': 'NQF level 3 or above',
                                'pt_hiqual_nqf2': 'NQF level 2 or above',
                                'country_name': 'ITL level 1',
                                'time_period': 'Year'},
                                inplace=True)
# create new rows for regions
education_df_2019 = education_df_2019.append(pd.DataFrame({'ITL level 1': regions}), ignore_index=True)
# set values for NQF level 4 or above for regions with England value
education_df_2019.loc[education_df_2019['ITL level 1'].isin(regions), 'NQF level 4 or above'] = education_df_2019.loc[education_df_2019['ITL level 1'] == 'England', 'NQF level 4 or above'].values[0]
# set values for NQF level 3 or above for regions with England value
education_df_2019.loc[education_df_2019['ITL level 1'].isin(regions), 'NQF level 3 or above'] = education_df_2019.loc[education_df_2019['ITL level 1'] == 'England', 'NQF level 3 or above'].values[0]
# set values for NQF level 2 or above for regions with England value
education_df_2019.loc[education_df_2019['ITL level 1'].isin(regions), 'NQF level 2 or above'] = education_df_2019.loc[education_df_2019['ITL level 1'] == 'England', 'NQF level 2 or above'].values[0]
# add year column
education_df_2019['Year'] = 2019


education_df_2019


/var/folders/j9/0qd6cw014pgb7rk0v_28prrh0000gn/T/ipykernel_15876/1205244664.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  education_df_2019 = education_df_2019.append(pd.DataFrame({'ITL level 1': regions}), ignore_index=True)


,Year,ITL level 1,NQF level 4 or above,NQF level 3 or above,NQF level 2 or above
0,2019,England,44.0,64.0,82.0
1,2019,Wales,40.0,60.0,79.0
2,2019,Northern Ireland,40.0,59.0,78.0
3,2019,Scotland,51.0,71.0,82.0
4,2019,London,44.0,64.0,82.0
5,2019,Yorkshire and The Humber,44.0,64.0,82.0
6,2019,South West,44.0,64.0,82.0
7,2019,East of England,44.0,64.0,82.0
8,2019,South East,44.0,64.0,82.0
9,2019,West Midlands,44.0,64.0,82.0


In [166]:
# read the csv file
education_df_2020 = pd.read_csv('data/Education/year_2020.csv')
# rename columns
education_df_2020.rename(columns={'pt_hiqual_nqf4': 'NQF level 4 or above',
                                'pt_hiqual_nqf3': 'NQF level 3 or above',
                                'pt_hiqual_nqf2': 'NQF level 2 or above',
                                'country_name': 'ITL level 1',
                                'time_period': 'Year'},
                                inplace=True)
# create new rows for regions
education_df_2020 = education_df_2020.append(pd.DataFrame({'ITL level 1': regions}), ignore_index=True)
# set values for NQF level 4 or above for regions with England value
education_df_2020.loc[education_df_2020['ITL level 1'].isin(regions), 'NQF level 4 or above'] = education_df_2020.loc[education_df_2020['ITL level 1'] == 'England', 'NQF level 4 or above'].values[0]
# set values for NQF level 3 or above for regions with England value
education_df_2020.loc[education_df_2020['ITL level 1'].isin(regions), 'NQF level 3 or above'] = education_df_2020.loc[education_df_2020['ITL level 1'] == 'England', 'NQF level 3 or above'].values[0]
# set values for NQF level 2 or above for regions with England value
education_df_2020.loc[education_df_2020['ITL level 1'].isin(regions), 'NQF level 2 or above'] = education_df_2020.loc[education_df_2020['ITL level 1'] == 'England', 'NQF level 2 or above'].values[0]
# add year column
education_df_2020['Year'] = 2020


education_df_2020

/var/folders/j9/0qd6cw014pgb7rk0v_28prrh0000gn/T/ipykernel_15876/2122461108.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  education_df_2020 = education_df_2020.append(pd.DataFrame({'ITL level 1': regions}), ignore_index=True)


,Year,ITL level 1,NQF level 4 or above,NQF level 3 or above,NQF level 2 or above
0,2020,England,47.0,66.0,83.0
1,2020,Wales,41.0,61.0,81.0
2,2020,Northern Ireland,44.0,63.0,80.0
3,2020,Scotland,52.0,71.0,82.0
4,2020,London,47.0,66.0,83.0
5,2020,Yorkshire and The Humber,47.0,66.0,83.0
6,2020,South West,47.0,66.0,83.0
7,2020,East of England,47.0,66.0,83.0
8,2020,South East,47.0,66.0,83.0
9,2020,West Midlands,47.0,66.0,83.0


In [167]:
# Concatenate all the education dataframes
education_df = pd.concat([education_df_2010, education_df_2012, education_df_2013, education_df_2014, education_df_2015, education_df_2016, education_df_2017, education_df_2018, education_df_2019, education_df_2020])
education_df

,ITL level 1,NQF level 4 or above,NQF level 3 or above,NQF level 2 or above,Year
0,England,38.0,59.0,78.0,2010
1,Wales,32.0,53.0,74.0,2010
2,Scotland,37.0,58.0,77.0,2010
3,Northern Ireland,31.0,51.0,71.0,2010
4,London,38.0,59.0,78.0,2010
...,...,...,...,...,...
8,South East,47.0,66.0,83.0,2020
9,West Midlands,47.0,66.0,83.0,2020
10,North West,47.0,66.0,83.0,2020
11,North East,47.0,66.0,83.0,2020


In [168]:
# add rows for Year 2011 for all the regions and set values to NaN
education_df = education_df.append(pd.DataFrame({'ITL level 1': regions, 'NQF level 4 or above': np.nan, 'NQF level 3 or above': np.nan, 'NQF level 2 or above': np.nan, 'Year': 2011}), ignore_index=True)
# add rows for regions with values 'England', 'Scotland', 'Wales' and 'Northern Ireland for Year 2011 and set values to 0
education_df = education_df.append(pd.DataFrame({'ITL level 1': ['England', 'Scotland', 'Wales', 'Northern Ireland'], 'NQF level 4 or above': np.nan, 'NQF level 3 or above': np.nan, 'NQF level 2 or above': np.nan, 'Year': 2011}), ignore_index=True)

/var/folders/j9/0qd6cw014pgb7rk0v_28prrh0000gn/T/ipykernel_15876/2488834413.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  education_df = education_df.append(pd.DataFrame({'ITL level 1': regions, 'NQF level 4 or above': np.nan, 'NQF level 3 or above': np.nan, 'NQF level 2 or above': np.nan, 'Year': 2011}), ignore_index=True)
/var/folders/j9/0qd6cw014pgb7rk0v_28prrh0000gn/T/ipykernel_15876/2488834413.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  education_df = education_df.append(pd.DataFrame({'ITL level 1': ['England', 'Scotland', 'Wales', 'Northern Ireland'], 'NQF level 4 or above': np.nan, 'NQF level 3 or above': np.nan, 'NQF level 2 or above': np.nan, 'Year': 2011}), ignore_index=True)


In [169]:
# show the education_df dataframe where Year is 2011
education_df[education_df['Year'] == 2011]

,ITL level 1,NQF level 4 or above,NQF level 3 or above,NQF level 2 or above,Year
130,London,NaN,NaN,NaN,2011
131,Yorkshire and The Humber,NaN,NaN,NaN,2011
132,South West,NaN,NaN,NaN,2011
133,East of England,NaN,NaN,NaN,2011
134,South East,NaN,NaN,NaN,2011
135,West Midlands,NaN,NaN,NaN,2011
136,North West,NaN,NaN,NaN,2011
137,North East,NaN,NaN,NaN,2011
138,East Midlands,NaN,NaN,NaN,2011
139,England,NaN,NaN,NaN,2011


In [170]:
# backfill the values for all the regions for Year 2011
education_df.loc[education_df['Year'] == 2011, 'NQF level 4 or above'] = education_df.loc[education_df['Year'] == 2010, 'NQF level 4 or above'].values
education_df.loc[education_df['Year'] == 2011, 'NQF level 3 or above'] = education_df.loc[education_df['Year'] == 2010, 'NQF level 3 or above'].values
education_df.loc[education_df['Year'] == 2011, 'NQF level 2 or above'] = education_df.loc[education_df['Year'] == 2010, 'NQF level 2 or above'].values

# show the education_df dataframe where Year is 2011
education_df[education_df['Year'] == 2011]

,ITL level 1,NQF level 4 or above,NQF level 3 or above,NQF level 2 or above,Year
130,London,38.0,59.0,78.0,2011
131,Yorkshire and The Humber,32.0,53.0,74.0,2011
132,South West,37.0,58.0,77.0,2011
133,East of England,31.0,51.0,71.0,2011
134,South East,38.0,59.0,78.0,2011
135,West Midlands,38.0,59.0,78.0,2011
136,North West,38.0,59.0,78.0,2011
137,North East,38.0,59.0,78.0,2011
138,East Midlands,38.0,59.0,78.0,2011
139,England,38.0,59.0,78.0,2011


In [171]:
# Convert the multi-index DataFrame to a dictionary
education_dict = education_df.set_index(['ITL level 1', 'Year'])[['NQF level 4 or above', 'NQF level 3 or above', 'NQF level 2 or above']].to_dict()

# Apply the dictionary to the all_vehicles_electric__df DataFrame by breaking the dataframe into chunks
all_vehicles_electric__df['NQF level 4 or above'] = all_vehicles_electric__df.set_index(['ITL level 1', 'Year']).index.map(education_dict['NQF level 4 or above']).fillna(0).astype(int)
all_vehicles_electric__df['NQF level 3 or above'] = all_vehicles_electric__df.set_index(['ITL level 1', 'Year']).index.map(education_dict['NQF level 3 or above']).fillna(0).astype(int)
all_vehicles_electric__df['NQF level 2 or above'] = all_vehicles_electric__df.set_index(['ITL level 1', 'Year']).index.map(education_dict['NQF level 2 or above']).fillna(0).astype(int)



In [172]:
# drop duplicates
all_vehicles_electric__df.drop_duplicates(inplace=True)
# keep rows where BodyType is 'Total'
all_vehicles_electric__df = all_vehicles_electric__df[all_vehicles_electric__df['BodyType'] == 'Cars']
# keep rows where Keepership is 'Total'
all_vehicles_electric__df = all_vehicles_electric__df[all_vehicles_electric__df['Keepership'] == 'Total']


In [173]:
all_vehicles_electric__df

,ITL level 3,Year,gdhi,sex,age,population,ITL level 1,ITL level 2,Average variable unit price (£/kWh),ULSP: Pump price (p/litre),ULSD: Pump price (p/litre),BodyType,Fuel,Keepership,Number of vehicles,Quarter,Sum of Chargepoints,NQF level 4 or above,NQF level 3 or above,NQF level 2 or above
10,Aberdeen City and Aberdeenshire,2010,18736.0,1,0,2801,Scotland,North Eastern Scotland,0.117261,116.904146,119.234615,Cars,Total,Total,0,1,0,37,58,77
52,Aberdeen City and Aberdeenshire,2010,18736.0,1,0,2801,Scotland,North Eastern Scotland,0.117261,116.904146,119.234615,Cars,Total,Total,0,2,0,37,58,77
94,Aberdeen City and Aberdeenshire,2010,18736.0,1,0,2801,Scotland,North Eastern Scotland,0.117261,116.904146,119.234615,Cars,Total,Total,0,3,0,37,58,77
136,Aberdeen City and Aberdeenshire,2010,18736.0,1,0,2801,Scotland,North Eastern Scotland,0.117261,116.904146,119.234615,Cars,Total,Total,0,4,0,37,58,77
178,Aberdeen City and Aberdeenshire,2010,18736.0,1,1,2762,Scotland,North Eastern Scotland,0.117261,116.904146,119.234615,Cars,Total,Total,0,1,0,37,58,77
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64240076,West Northamptonshire,2020,22354.0,2,89,451,East Midlands,"Leicestershire, Rutland and Northamptonshire",0.166876,114.092268,119.455858,Cars,Total,Total,2235,4,0,47,66,83
64240097,West Northamptonshire,2020,22354.0,2,90,2140,East Midlands,"Leicestershire, Rutland and Northamptonshire",0.166876,114.092268,119.455858,Cars,Total,Total,1540,1,0,47,66,83
64240118,West Northamptonshire,2020,22354.0,2,90,2140,East Midlands,"Leicestershire, Rutland and Northamptonshire",0.166876,114.092268,119.455858,Cars,Total,Total,1618,2,0,47,66,83
64240139,West Northamptonshire,2020,22354.0,2,90,2140,East Midlands,"Leicestershire, Rutland and Northamptonshire",0.166876,114.092268,119.455858,Cars,Total,Total,2053,3,0,47,66,83


In [174]:
# final dataframe for chargepoints, gdhi, population, ownership, sex, age to do analysis
all_vehicles_electric__df
# save the dataframe as a porquet file
all_vehicles_electric__df.to_parquet('data/Cleaned data/all_vehicles_electric__df.parquet', engine='pyarrow')

In [175]:
# read in government_policy csv
government_policy_df = pd.read_csv('data/government_policy.csv')
government_policy_df
# delete columns
government_policy_df.drop(['Source', 'Country','Region','Scope'], axis=1, inplace=True)
# create a new column called Fuel Type and set it to 'Electric' for rows where Category contains "EV"
government_policy_df.loc[government_policy_df['Category'].str.contains('EV', case=False, na=False), 'Fuel'] = 'electric'
# Create a new dataframe with the desired fuel types
fuel_types_df = pd.DataFrame({
    'Fuel2': ['Battery electric', 'Plug-in hybrid electric (diesel)', 'Plug-in hybrid electric (petrol)',
             'Range extended electric', 'Hybrid electric (petrol)']
})

fuel_types_df["Fuel"] = "electric"

# Merge the fuel_types_df with government_policy_df
government_policy_df = government_policy_df.merge(fuel_types_df, how='left', on='Fuel')

# if fuel is 'electric' then make multiple rows for fuel2 where values are 'Battery electric', 'Plug-in hybrid electric (diesel)', 'Plug-in hybrid electric (petrol)', 'Range extended electric' and 'Hybrid electric (petrol)'

# set the row to Diesel for rows where Category contains "DV"
government_policy_df.loc[government_policy_df['Category'].str.contains('DV', case=False, na=False), 'Fuel'] = 'diesel'
# set the row to "Light goods vehicles" for rows were Category contains "LDV"
government_policy_df.loc[government_policy_df['Category'].str.contains('LDV', case=False, na=False), 'BodyType'] = 'Light goods vehicles'
# set the row to "Heavy goods vehicles" for rows were Category contains "HDV"
government_policy_df.loc[government_policy_df['Category'].str.contains('HDV', case=False, na=False), 'BodyType'] = 'Heavy goods vehicles'
# set the row to "Buses and coaches" for rows were Category contains "Bus"
government_policy_df.loc[government_policy_df['Category'].str.contains('Bus', case=False, na=False), 'BodyType'] = 'Buses and coaches'
# drop Fuel column
government_policy_df.drop(['Fuel'], axis=1, inplace=True)
# rename Fuel2 to Fuel
government_policy_df.rename(columns={'Fuel2': 'Fuel'}, inplace=True)

government_policy_df.dropna(subset=['Fuel', 'BodyType'], how='all', inplace=True)
government_policy_df

,Type,Description,Category,Year,Fuel,BodyType
0,Ambition,"30% ZEV sales in M/HDVs by 2030, 100% by 2040",M/HDV,2021,NaN,Heavy goods vehicles
2,Ambition,Electric vehicle infrastructure strategy: Arou...,EVSE,2022,Battery electric,NaN
3,Ambition,Electric vehicle infrastructure strategy: Arou...,EVSE,2022,Plug-in hybrid electric (diesel),NaN
4,Ambition,Electric vehicle infrastructure strategy: Arou...,EVSE,2022,Plug-in hybrid electric (petrol),NaN
5,Ambition,Electric vehicle infrastructure strategy: Arou...,EVSE,2022,Range extended electric,NaN
6,Ambition,Electric vehicle infrastructure strategy: Arou...,EVSE,2022,Hybrid electric (petrol),NaN
7,Ambition,"30% ZEV sales in M/HDVs by 2030, 100% by 2040",M/HDV,2021,NaN,Heavy goods vehicles
9,Proposal,End the sale of non-ZEV HDVs under 26t gross v...,M/HDV,2021,NaN,Heavy goods vehicles
12,Legislation,Grant schemes for EV charging infrastructure i...,EVSE,2016,Battery electric,NaN
13,Legislation,Grant schemes for EV charging infrastructure i...,EVSE,2016,Plug-in hybrid electric (diesel),NaN


In [176]:
vehicle_df['Fuel'].unique()

array(['Battery electric', 'Total', 'Plug-in hybrid electric (diesel)',
       'Plug-in hybrid electric (petrol)', 'Range extended electric'],
      dtype=object)

In [177]:
# create a dictionary mapping 'Year','Fuel Type' and 'BodyType' to 'Description' in government_policy_electric_df
policy_dict = government_policy_df.set_index(['Year','Fuel'])['Description'].to_dict()
weight_dict = government_policy_df.set_index(['Year','BodyType'])['Description'].to_dict()
# update the 'Government Policy' column in vehicle_df based on the dictionary where Fuel contains strings from Fuel
vehicle_df['Government Policy_EV'] = vehicle_df.set_index(['Year','Fuel']).index.map(policy_dict).fillna('No policy')
vehicle_df['Government Policy_weight'] = vehicle_df.set_index(['Year','BodyType']).index.map(weight_dict).fillna('No policy')
# drop duplicates
vehicle_df.drop_duplicates(inplace=True)
# get all values in Year column in all_vehicles_electric__df
vehicle_df


,Year,Units,BodyType,Fuel,Keepership,ONS Sort,ONS Code,ONS Geography,Number of vehicles,Quarter,Government Policy_EV,Government Policy_weight
0,2009,Number,Buses and coaches,Battery electric,Company,1.0,K02000001,United Kingdom,0,4,No policy,No policy
1,2009,Number,Buses and coaches,Battery electric,Company,2.0,K03000001,Great Britain,38,4,No policy,No policy
2,2009,Number,Buses and coaches,Battery electric,Company,3.0,E92000001,England,36,4,No policy,No policy
3,2009,Number,Buses and coaches,Battery electric,Company,4.0,E12000001,North East,0,4,No policy,No policy
4,2009,Number,Buses and coaches,Battery electric,Company,5.0,E06000047,County Durham,0,4,No policy,No policy
...,...,...,...,...,...,...,...,...,...,...,...,...
1719063,2022,Number,Total,Total,Total,460.0,N09000010,"Newry, Mourne and Down",693,3,No policy,Electric vehicle infrastructure strategy: Arou...
1719064,2022,Number,Total,Total,Total,461.0,[z],Local Authority unknown within Northern Ire...,761,3,No policy,Electric vehicle infrastructure strategy: Arou...
1719065,2022,Number,Total,Total,Total,462.0,[z],"Vehicle under disposal, previously GB",15301,3,No policy,Electric vehicle infrastructure strategy: Arou...
1719066,2022,Number,Total,Total,Total,463.0,[z],"Vehicle under disposal, previously NI",191,3,No policy,Electric vehicle infrastructure strategy: Arou...


In [178]:
vehicle_df['BodyType'].unique()
# save the dataframe to a porquet file
vehicle_df.to_parquet('data/Cleaned data/vehicle_policy_df.parquet', engine='pyarrow')


In [179]:
vehicle_df

,Year,Units,BodyType,Fuel,Keepership,ONS Sort,ONS Code,ONS Geography,Number of vehicles,Quarter,Government Policy_EV,Government Policy_weight
0,2009,Number,Buses and coaches,Battery electric,Company,1.0,K02000001,United Kingdom,0,4,No policy,No policy
1,2009,Number,Buses and coaches,Battery electric,Company,2.0,K03000001,Great Britain,38,4,No policy,No policy
2,2009,Number,Buses and coaches,Battery electric,Company,3.0,E92000001,England,36,4,No policy,No policy
3,2009,Number,Buses and coaches,Battery electric,Company,4.0,E12000001,North East,0,4,No policy,No policy
4,2009,Number,Buses and coaches,Battery electric,Company,5.0,E06000047,County Durham,0,4,No policy,No policy
...,...,...,...,...,...,...,...,...,...,...,...,...
1719063,2022,Number,Total,Total,Total,460.0,N09000010,"Newry, Mourne and Down",693,3,No policy,Electric vehicle infrastructure strategy: Arou...
1719064,2022,Number,Total,Total,Total,461.0,[z],Local Authority unknown within Northern Ire...,761,3,No policy,Electric vehicle infrastructure strategy: Arou...
1719065,2022,Number,Total,Total,Total,462.0,[z],"Vehicle under disposal, previously GB",15301,3,No policy,Electric vehicle infrastructure strategy: Arou...
1719066,2022,Number,Total,Total,Total,463.0,[z],"Vehicle under disposal, previously NI",191,3,No policy,Electric vehicle infrastructure strategy: Arou...
